#PreProcess

In [ ]:
!pip install csaps

In [ ]:
import pandas as pd
import numpy as np
import scipy as sc
from csaps import csaps  # for cubic smoothing splines
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed


def spline_calculator(t, y, smooth, refit, n_jobs):
    """Does cubic smooth spline approximation on a 1d dataset.

    Args:
        t: The time array.
        y: The 1D dataset array.
        smooth: The smoothing factor or regularization parameter.
        refit: Boolean flag indicating whether to fit the smoothing spline on the entire data.
        n_jobs: Number of cores to be used for parallelization.

    Returns:
        The cubic smoothing spline.

    """
    if isinstance(smooth, (int, float)):
        spline = csaps(t, y, smooth=smooth)

    else:
        t_train, t_test, y_train, y_test = train_test_split(
            t[1:-1], y[1:-1], test_size=0.3, random_state=42
        )

        t_train = np.append(t_train, [t[0], t[-1]])
        y_train = np.append(y_train, [y[0], y[-1]])

        merge_train = list(zip(t_train, y_train))
        merge_test = list(zip(t_test, y_test))
        merge_train.sort(key=lambda x: x[0])
        merge_test.sort(key=lambda x: x[0])

        t_train, y_train = zip(*merge_train)
        t_test, y_test = zip(*merge_test)

        t_train = np.array(t_train)
        y_train = np.array(y_train)
        t_test = np.array(t_test)
        y_test = np.array(y_test)

        param_grid = np.linspace(0, 1, 101)

        err = float("inf")
        results = Parallel(n_jobs=n_jobs)(
            delayed(spline_error)(t_train, y_train, t_test, y_test, param)
            for param in param_grid
        )

        for result in results:
            param, curr_err = result
            if curr_err < err:
                err = curr_err
                best_lambda = param

        if refit:
            spline = csaps(t, y, smooth=best_lambda)

        else:
            spline = csaps(t_train, y_train, smooth=best_lambda)

    return spline


def spline_error(t_train, y_train, t_test, y_test, param):
    """Calculates the error for a given smoothing parameter.

    Args:
        t_train: The time array of the training data.
        y_train: The training dataset array.
        t_test: The time array of the test data.
        y_test: The test dataset array.
        param: The smoothing parameter.

    Returns:
        Tuple containing the parameter value and the error.

    """
    spline = csaps(t_train, y_train, smooth=param)
    ytest_pred = spline(t_test)
    error = mean_squared_error(y_test, ytest_pred)
    return (param, error)


def cubic_smooth(t, y, smooth, refit, n_jobs=-1):
    """Does cubic smoothing spline on a 2d dataset fitting a spline to each column.

    Args:
        t: The time array.
        y: The 2D dataset array.
        smooth: The smoothing factor or regularization parameter.
        refit: Boolean flag indicating whether to fit the smoothing spline on the entire data.
        n_jobs: Number of cores to be used. Default is -1 (all available cores).

    Returns:
        The final spline function and its derivative.

    """
    if len(y.shape) == 1:
        final_spline = spline_calculator(t, y, smooth, refit, n_jobs)
        final_derivative = final_spline.spline.derivative(nu=1)

    elif len(y.shape) == 2:
        splines = [
            spline_calculator(t, y[:, i], smooth, refit, n_jobs)
            for i in range(y.shape[1])
        ]
        final_spline = lambda x: np.array(  # noqa: E731
            [spline(x) for spline in splines]
        ).T
        final_derivative = lambda x: np.array(  # noqa: E731
            [spline.spline.derivative(nu=1)(x) for spline in splines]  # noqa: E731
        ).T

    else:
        raise ValueError(
            "y should be of shape Nx1 or Nxm for which m splines would be \
                fit. Cannot be more than 2-dimensions."
        )
    return final_spline, final_derivative


def read_file(file_path):
    """Reads a file from various formats using pandas library in Python.

    Args:
        file_path: The path of the file to be read.

    Returns:
        A pandas DataFrame object.

    """
    file_extension = file_path.split(".")[-1]
    if file_extension == "csv":
        df = pd.read_csv(file_path, index_col=0)
    elif file_extension == "xls" or file_extension == "xlsx":
        df = pd.read_excel(file_path, index_col=0, header=None)
    elif file_extension == "json":
        df = pd.read_json(file_path)
        df = df.set_index(df.columns[0])
    elif file_extension == "txt":
        df = pd.read_table(file_path, header=None, index_col=0)
    elif file_extension == "tsv":
        df = pd.read_csv(file_path, sep="\t", header=None, index_col=0)
    else:
        raise ValueError(
            "Unsupported file format. Please provide a CSV, Excel, JSON, TSV or Text file."
        )
    return df


def num_params(f, args, max_lim=1000):
    """Returns the number of parameters in a function.

    Args:
        f: The function for which the number of arguments needs to be found.
        args: The arguments for the function.
        max_lim: The maximum number of parameters to consider. Default is 1000.

    Returns:
        The number of parameters used in the function.

    """
    for i in range(max_lim):
        temp = np.ones(i)
        try:
            f(args, temp)
            return len(temp)
        except:  # noqa: E722
            pass

    raise ValueError(f"Number of Parameters exceed {max_lim}")


def baseline_shift(values):
    """Performs baseline shift if values contain negative values.

    Args:
        values: The array of values.

    Returns:
        The shifted values.

    """
    mini = np.min(values, axis=0)
    add_ = np.abs(np.minimum(mini, np.array([0] * len(mini))))
    values_new = values + add_
    return values_new


def threshold_cutoff(values, threshold):
    """Replaces values less than the threshold with the threshold value.

    Args:
        values: The array of values.
        threshold: The threshold value.

    Returns:
        The modified values.

    """
    values[values < threshold] = threshold
    return values


def savgol_filter(
    values,
    window_length,
    polyorder,
    deriv=0,
    delta=1.0,
    axis=0,
    mode="interp",
    cval=0.0,
):
    """Applies the Savitzky-Golay filter to the values.

    Args:
        values: The array of values.
        window_length: The length of the window for filtering.
        polyorder: The order of the polynomial to fit.
        deriv: The order of the derivative to compute. Default is 0.
        delta: The sampling interval. Default is 1.0.
        axis: The axis along which to apply the filter. Default is 0.
        mode: The extrapolation mode. Default is "interp".
        cval: The constant value for boundary extrapolation. Default is 0.0.

    Returns:
        The filtered values.

    """
    r = sc.signal.savgol_filter(
        values, window_length, polyorder, deriv, delta, axis=axis, mode=mode, cval=cval
    )
    return r

# RateLawGen

In [ ]:
from typing import Any
from collections.abc import Callable
import numpy as np
import ast
from copy import deepcopy


def powerlaw_generator(N, d, idx):
    """Generates a combination of rate laws in terms of power given a stoichiometric list.

    Args:
        N: The stoichiometric list (all elements should be greater than or equal to zero).
        d: An empty dictionary.
        idx: Refers to the index of the rate constant to be used.
             0 for irreversible reactions, 1 for reversible reactions.

    Returns:
        None. Modifies the dictionary in place.

    Uses recursion and hashmap (dynamic programming) for efficiency.
    """
    s = str(N)
    if s in d:
        return d[s]

    elif sum(N) == 0:
        d[s] = lambda y, K: K[idx] * np.ones((y[0].shape))
        return d[s]

    else:
        for i in range(len(N)):
            if N[i] != 0:
                Nnew = deepcopy(N)
                Nnew[i] = Nnew[i] - 1
                powerlaw_generator(Nnew, d, idx)
                if s not in d:
                    fun = d[str(Nnew)]
                    # binding i is important. Else will return the same function
                    d[s] = lambda y, K, saved_i=i: y[saved_i] * fun(y, K)
        return d[s]


class RateLaw:
    def __init__(self, type: str, expression: str, function: Callable[[list], Any]):
        self.type = type
        self.expression = expression
        self.function = function

    def __repr__(self):
        return f'RateLaw(type = "{self.type}", Expression = "{self.expression}")'

    # have to cross check error handling capabilities #
    def __add__(self, other):
        """Addition compatibility"""
        func = lambda y, K: self.function(y, K) + other.function(y, K)  # noqa: E731
        return RateLaw(
            type="<unk>",
            expression=self.expression + "+ " + other.expression,
            function=func,
        )

    def __sub__(self, other):
        """Subtraction compatibility"""
        func = lambda y, K: self.function(y, K) - other.function(y, K)  # noqa: E731
        return RateLaw(
            type="<unk>",
            expression=self.expression + "- " + other.expression,
            function=func,
        )

    def __mul__(self, other):
        """Product compatibility"""
        func = lambda y, K: self.function(y, K) * other.function(y, K)  # noqa: E731
        return RateLaw(
            type="<unk>",
            expression=self.expression + "*" + other.expression,
            function=func,
        )

    def __truediv__(self, other):
        """Division compatibility"""
        func = lambda y, K: self.function(y, K) / other.function(y, K)  # noqa: E731
        return RateLaw(
            type="<unk>",
            expression=self.expression + "/ " + other.expression,
            function=func,
        )


class CandidateRateLaws:
    """
    Represents a collection of candidate rate laws based on stoichiometric coefficients.

    Args:
        N (list): The stoichiometric list.
        type (list): The types of rate laws to consider (e.g., ["Reversible", "Irreversible"]).

    Attributes:
        _ratelaws (list): The list of RateLaw objects representing the candidate rate laws.

    """

    def __init__(self, N, type=["Reversible", "Irreversible"]):
        self.N = N

        if (
            "Irreversible" in type or "Reversible" in type
        ):  # This might misbehave if user types IrReversible. Can Improvise
            # we have to calculate irreversible rate laws even if it is just reversible
            N_irr = [abs(num) if num < 0 else 0 for num in N]
            d_irr = {}
            powerlaw_generator(N_irr, d_irr, idx=0)

            # irreversible
            self._ratelaws = []
            for key in d_irr.keys():
                temp = ast.literal_eval(
                    key
                )  # converts string repr of list to list back
                exp_list = [f"C[{i}]^{num}" for i, num in enumerate(temp) if num > 0]
                exp_list.insert(0, "K[0]")
                exp = "*".join(exp_list)

                if exp == "":
                    exp == "<unk>"  # unknown expression if it's unable to generate

                if "Irreversible" in type:
                    self._ratelaws.append(
                        RateLaw(
                            type="Irreversible", expression=exp, function=d_irr[key]
                        )
                    )

        if "Reversible" in type:
            N_rev_temp = [num if num > 0 else 0 for num in N]
            d_rev_temp = {}
            powerlaw_generator(N_rev_temp, d_rev_temp, idx=1)

            for key1 in d_irr.keys():
                for key2 in d_rev_temp.keys():
                    temp1 = ast.literal_eval(key1)
                    temp2 = ast.literal_eval(key2)
                    exp_list1 = [
                        f"C[{i}]^{num}" for i, num in enumerate(temp1) if num > 0
                    ]
                    exp_list1.insert(0, "K[0]")
                    exp_list2 = [
                        f"C[{i}]^{num}" for i, num in enumerate(temp2) if num > 0
                    ]
                    exp_list2.insert(0, "K[1]")
                    exp_1 = "*".join(exp_list1)
                    exp_2 = "*".join(exp_list2)

                    if exp_1 == "" and exp_2 == "":
                        exp = "<unk>"
                    elif exp_2 == "":
                        exp = exp_1

                    if exp_1 == "K[0]" or exp_2 == "K[1]":
                        continue

                    r_fun = lambda y, K, saved_key1=key1, saved_key2=key2: d_irr[  # noqa: E731
                        saved_key1
                    ](y, K) - d_rev_temp[saved_key2](y, K)
                    self._ratelaws.append(
                        RateLaw(
                            type="Reversible",
                            expression=exp_1 + "- " + exp_2,
                            function=r_fun,
                        )
                    )

    def __len__(self):
        """
        Returns the number of candidate rate laws.

        Returns:
            int: The number of candidate rate laws.

        """
        return len(self._ratelaws)

    def __getitem__(self, position):
        """
        Returns the candidate rate law at the specified position.

        Args:
            position (int): The position of the candidate rate law.

        Returns:
            RateLaw: The candidate rate law at the specified position.

        """

        return self._ratelaws[position]

    def __repr__(self) -> str:
        """
        Returns a string representation of the CandidateRateLaws object.

        Returns:
            str: The string representation of the CandidateRateLaws object.

        """
        return "[" + "\n".join(list(map(str, self._ratelaws))) + "]"

    def append(self, rl: RateLaw) -> None:
        """
        Appends a RateLaw object to the list of candidate rate laws.

        Args:
            rl (RateLaw): The RateLaw object to append.

        Returns:
            None

        """
        self._ratelaws.append(rl)

    @property
    def ratelaws(self):
        """
        Returns a list of rate law functions.

        Returns:
            list: The list of rate law functions.

        """
        rl = [self._ratelaws[i].function for i in range(len(self._ratelaws))]
        return rl

#Data Tools

In [ ]:
import pandas as pd
import numpy as np
import scipy as sc
# from preprocess import cubic_smooth, baseline_shift, read_file
# from preprocess import threshold_cutoff, savgol_filter
import warnings
import textwrap
import ast
from IPython.display import display, Markdown
from scipy.integrate import odeint
from scipy import interpolate

class DataTools:

    amax = 5 # Datatools class attribute. It is inherited by child class Incremental
             # Max. allowed attempts for incorrect data entered by user

    def __init__(self):
        self.N = None   # Stoichiometric Matrix
        self.Na = None  # Stoichiometric Matrix (available species)
        self.Nu = None  # Stoichiometric Matrix (unavailable species)
        self.R = None   # Num Reactions
        self.S = None   # Num Species
        self.Mw = None  # Mol Weight kg/kmol
        self.V = None   # volume
        self.Winhat = None # mass frac composition of inlet
        self.Win = None # inlet composition
        self.Wina = None # inlet composition (available species)
        self.Winu = None # inlet composition (unavailable species)
        self.P = None # no of inlets
        self.uin = None # inlet flowrate (kg/time)
        self.uout = None # outlet flowrate (kg/time)
        self.n0 = None # initial no of moles
        self.n0a = None # initial no of moles (available species)
        self.n0u = None # initial no of moles (unavailable species)
        self.time = None
        self.methodology = None


    def add_reactor_config_flag(self):

        str_0a = ("Enter reactor configuration as a number. Permitted entries: 1 = Batch; 2 = Semi-Batch;"
                " 3 = CSTR.")
        str_0b = "Enter reactor configuration:"
        str_r = "Enter reactor configuration again:"

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(str_0a)
                self.rc_flag = input(str_0b)
            else:
                self.rc_flag = input(str_r)

            print("Entered reactor configuration is:", self.rc_flag)

            try:
                self.rc_flag = ast.literal_eval(self.rc_flag) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.rc_flag,int): # rc_flag is not an integer
                print("Entered object is not an integer. Reactor configuration must be an integer.")
                continue

            # No need to check for np.nan due to this check
            if isinstance(self.rc_flag,int) and self.rc_flag != 1 and self.rc_flag != 2 and self.rc_flag != 3:
                print("Invalid value of reactor configuration has been entered. Allowed values are: 1 = Batch; 2 = Semi-Batch; 3 = CSTR.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for reactor configuration received successfully.**"))
                break





    def add_number_reactions(self):

        str_0 = ("Enter the number of reactions in the reaction system:")
        str_r = "Enter the number of reactions in the reaction system again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                self.R = input(str_0)
            else:
                self.R = input(str_r)

            print("Entered value of number of reactions is:", self.R)

            try:
                self.R = ast.literal_eval(self.R) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.R,int): # self.R is not an integer
                print("Entered object is not an integer. Number of reactions must be an integer.")
                continue

            if isinstance(self.R,int) and self.R <= 0:
                print("Entered value is either zero or negative. Number of reactions must be a positive integer.")
                continue

            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan is an invalid python expression
            # "np.nan" converts it to string data type
            if np.isnan(self.R) == True:
                print("A NaN value has been entered. NaN value is not allowed.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for number of reactions received successfully.**"))
                break







    def add_number_species(self):

        str_0 = ("Enter the number of species in the reaction system:")
        str_r = "Enter the number of species in the reaction system again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                self.S = input(str_0)
            else:
                self.S = input(str_r)

            print("Entered value of number of species is:", self.S)

            try:
                self.S = ast.literal_eval(self.S) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.S,int): # self.rhov0_flag is not an integer
                print("Entered object is not an integer. Number of species must be an integer.")
                continue

            if isinstance(self.S,int) and self.S <= 0:
                print("Entered value is either zero or negative. Number of species must be a positive integer.")
                continue


            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan is an invalid python expression
            # "np.nan" converts it to string data type
            if np.isnan(self.S) == True:
                print("A NaN value has been entered. NaN value is not allowed.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for number of species received successfully.**"))
                break






    def add_stoichio(self):

        str_0a = ("Enter Stoichiometric Matrix as a two-dimensional list. For reaction system with "
                  "two reactions R1: A -> 2B and R2: 3 A -> 4 C, the stoichiometric matrix is "
                  "[[-1,2,0],[-3,0,4]]. In the stoichiometric matrix, the order of reactions is R1, R2 "
                  "and the order of species is A, B, C.")
        str_0b = "Enter Stoichiometric Matrix:"
        str_r = "Enter Stoichiometric Matrix again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                self.N = input(str_0b)
            else:
                self.N = input(str_r)

            print("Entered stoichiometric matrix is:", self.N)

            try:
                self.N = ast.literal_eval(self.N) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.N,list): # N is not a list
                print("Entered object is not a list.")
                continue

            try:
                self.N = np.array(self.N) # Convert N to numpy array
            except Exception as emsg: # List N could not be converted to numpy array
                print("Unable to convert input to numpy array, possibly due to inhomogeneous shape i.e. unequal number of columns across rows.")
                print("Further information:")
                print(emsg)
                continue

            if not (np.issubdtype(self.N.dtype,np.number)):
                print("Elements of stoichiometric matrix must be numbers.")
                continue

            if not np.all(np.isreal(self.N)):
                print("Some elements of stoichiometric matrix are complex.")
                continue

            if self.N.ndim != 2:
                print("Number of dimensions of stoichiometric matrix is not equal to 2.")
                continue

            if self.N.shape[0] != self.R:
                print(f"Number of rows of stoichiometric matrix is {self.N.shape[0]}, which is not equal to the number of reactions {self.R}, entered previously.")
                continue

            if self.N.shape[1] != self.S:
                print(f"Number of columns of stoichiometric matrix is {self.N.shape[1]}, which is not equal to the number of species {self.S}, entered previously.")
                continue

            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan as an element of N is an invalid python expression
            # "np.nan" as an element of N converts it to string data type
            # [[1,,]] is an invalid python expression
            if np.any(np.isnan(self.N)) == True:
                print("At least one element in the stoichiometric matrix is a NaN value. NaN values are not allowed.")
                continue

            if not all(np.any(self.N < 0, axis=1)) == True:
                print("At least one element in each row of the stoichiometric matrix must be negative.")
                continue

            if not all(np.any(self.N > 0, axis=1)) == True:
                print("At least one element in each row of the stoichiometric matrix must be positive.")
                continue

            # If at least one element in each row of the stoichiometric matrix is
            # negative/positive, the corresponding row is non-zero.
            # Hence, no need to check for a zero row.

            if np.any(np.all(self.N==0,axis=0)==True) == True:
                print("At least one column in the stoichiometric matrix is a column of zeros. Columns of zeros are not allowed.")
                continue

            # If some elements of stoichiometric matrix is/are type float, then all
            # integer elements are converted to type float. Hence the following logic
            # to check whether all negative elements are integers.

            Nn = self.N[self.N<0] # Extract all negative elements to a numpy array
            Nnr = np.array(list(map(int,Nn))) # Negative elements rounded to nearest integer
            dN = Nn - Nnr
            if not np.all(dN==0):
                print("All negative elements of stoichiometric matrix must be integers.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for stoichiometric matrix received successfully.**"))
                self.S_l = [chr(x) for x in range(65, 65 + self.S)] # ordered
                self.S_s = set(self.S_l) # unordered
                display(Markdown(f"**Reacting species have been assigned the following names: {self.S_l} in the same order in which they appear in the columns of the stoichiometric matrix.**"))
                break







    def add_rxn_rev(self):

        str_0a = ("Enter reaction reversibility information as a one-dimensional list. Enter 0 for "
                  "irreversible reaction and 1 for reversible reaction. For reaction system with 2 "
                  "reactions, where reaction 1 is irreversible and reaction 2 is reversible, enter [0,1]. "
                  "The order of reactions should be the same as the order in which they appear in the"
                  " rows of the stoichiometric matrix.")
        str_0b = "Enter reaction reversibility information:"
        str_r = "Enter reaction reversibility information again:"

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                self.rrv = input(str_0b)
            else:
                self.rrv = input(str_r)

            print("User input is:", self.rrv)

            try:
                self.rrv = ast.literal_eval(self.rrv) # Evaluate user input
                self.rrv = [int(x) for x in self.rrv]
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.rrv,list): # N is not a list
                print("Entered object is not a list.")
                continue

            if len(self.rrv) != self.N.shape[0]:
                print("The number of elements in the list is not equal to the number of rows of the stoichiometric matrix.")
                continue


            # This check negates the need to check for NaN values
            if not set(self.rrv).issubset({0,1}):
                print("Some elements of the list have values other than 0 and 1.")
                continue

            # Here, all checks have been satisfied: Elements of list self.rrv are 0/1 and len(self.rrv) = R = N.shape[0]

            if i <= (self.amax-1):
                display(Markdown("**Data for reaction reversibility received successfully.**"))
                self.rrv = list(map(lambda x: "Irreversible" if x == 0 else "Reversible", self.rrv))
                break







    def conc_data_struct_disp(self):

        display(Markdown("**• The following is pertinent information regarding the concentration data file structure.**"))
        print("• The first row in the file must contain column headers.")
        print("• Column 1 header can have any name.")
        print("• Other column headers (columns 2, 3, . . . ) must have species names.")
        print(f"• Allowed species names are: {self.S_l}")
        print("• The first column in the file must contain time points at which concentration data is collected.")
        print("• The first time point must be zero.")
        print("• The remaining columns pertain to concentration data of participating species.")
        print("• The file can be a csv file or a tab-separated text file.")
        print("• All data should be in SI units.")






    def add_concentration_data_from_file(self):

        str_0a = "Upload concentration data file to local storage and enter concentration data file name."
        str_0b = "Enter concentration data file name:"
        str_r = "Enter concentration data file name again:"
        str_max = "Maximum number of attempts exceeded. Please start again."

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                fname = input(str_0b)
            else:
                fname = input(str_r)

            print("Entered file name is:", fname)

            try:
                self.time_conc_df = pd.read_csv(fname) # Read file
            except Exception as emsg: # File not uploaded or some other error
                print(emsg)
                continue

            try:
                self.time_conc = np.array(self.time_conc_df) # Read file
            except Exception as emsg: # Dataframe could not be converted to numpy array
                print(emsg)
                continue

            if not np.issubdtype(self.time_conc.dtype, np.number):
                print("Time and concentration values must be numbers. Non-numeric data is present in concentration data file.")
                continue

            # Check for missing data

            if np.any(np.isnan(self.time_conc)) == True:
                print("At least one value in the concentration data file is missing. Missing values are not allowed.")
                continue

            if not np.all(self.time_conc >=0):
                print("All values must be non-negative. At least one value in the concentration data file is negative.")
                continue

            if not int(self.time_conc[0][0]) == 0:
                print("First time point must be zero. It is not equal to zero in the concentration data file.")
                continue

            if not np.all(np.diff(self.time_conc[:,0]) > 0):
                print("Time points must be strictly monotonously increasing. This is not the case in the concentration data file.")
                continue

            self.S_c = set(self.time_conc_df.columns[1:])

            # Species names extracted from the first row of concentration data file
            # are stored in the set S_c

            if not len(self.S_c) <= self.S:
                print(f"The number of species in the concentration data file is {len(self.S_c)}. It should be less than or equal to {self.S}, the number of species in the stoichiometric matrix.")
                continue

            if not self.S_c.issubset(self.S_s):
                sp_nms_all = ", ".join(sorted(self.S_l))
                sp_nms_cdf = ", ".join(sorted(self.S_c))
                print(f"One or more species names in concentration data file are invalid. Allowed species names are: {self.sp_nms_all}, whereas the species names in the concentration data file are {self.sp_nms_cdf}. ")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Time and concentration data received successfully.**"))
                self.t_vals = self.time_conc[:,0]
                self.c_vals = self.time_conc[:,1:]
                break






    def vol_data_struct_disp(self):

        display(Markdown("**• The following is pertinent information regarding the structure of the volume data file.**"))
        print("• Column headers are not allowed.")
        str_1 = ("• Column 1 in the file must contain time points at which the reactor volume data is "
                "collected. Column 2 in the file should contain the reactor volume data.")
        print(textwrap.fill(str_1,width=125))
        print("• The time points should be the same as those in the concentration data file.")
        print("• The file should be a csv file.")
        print("• All data should be in SI units.")





    def add_vol_data(self):

        str_0a = "Upload volume data file to local storage and enter volume data file name."
        str_0b = "Enter volume data file name:"
        str_r = "Enter volume data file name again:"
        str_max = "Maximum number of attempts exceeded. Please start again."

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                fname = input(str_0b)
            else:
                fname = input(str_r)

            print("Entered file name is:", fname)

            try:
                self.time_vol_df = pd.read_csv(fname,header=None) # Read file
            except Exception as emsg: # File not uploaded or some other error
                print(emsg)
                continue

            try:
                self.time_vol = np.array(self.time_vol_df) # Read file
            except Exception as emsg: # Dataframe could not be converted to numpy array
                print(emsg)
                continue

            if not np.issubdtype(self.time_vol.dtype, np.number):
                print("Time and volume values must be numbers. Non-numeric data is present in volume data file.")
                continue

            # Check for missing data

            if np.any(np.isnan(self.time_vol)) == True:
                print("At least one value in the volume data file is missing. Missing values are not allowed.")
                continue

            if not np.all(self.time_vol >=0):
                print("All values must be non-negative. At least one value in the volume data file is negative.")
                continue

            if not int(self.time_vol[0][0]) == 0:
                print("First time point must be zero. It is not equal to zero in the volume data file.")
                continue

            if not np.all(np.diff(self.time_vol[:,0]) > 0):
                print("Time points must be strictly monotonously increasing. This is not the case in the volume data file.")
                continue

            if not np.all(self.time_vol[:,0] == self.time_conc[:,0]) == True:
                print("Time data points in volume data file are not the same as those in the concentration data file.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Time and volume data received successfully.**"))
                self.V_vals = self.time_vol[:,1]
                break









    def add_vol_batch(self):

        # Add attributes V0, n0, volume data (if provided by user)

        str_0a = ("Is the batch reactor volume constant? Enter 1 if batch reactor volume is constant."
                  " Enter 2 if batch reactor volume is variable. For variable volume batch reactor,"
                  " time series reactor volume data needs to be provided in a csv file.")
        str_0b = "Enter requested information:"
        str_r = "Enter requested information again:"

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(str_0a)
                self.brv_flag = input(str_0b)
            else:
                self.brv_flag = input(str_r)

            print("Entered information is:", self.brv_flag)

            try:
                self.brv_flag = ast.literal_eval(self.brv_flag) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.brv_flag,int): # brv_flag is not an integer
                print("Entered object is not an integer. Only integer values can be entered.")
                continue

            # No need to check for np.nan due to this check
            if isinstance(self.brv_flag,int) and self.brv_flag != 1 and self.brv_flag != 2:
                print("Invalid value has been entered. Allowed values are 1 and 2")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data regarding batch reactor volume variability received successfully.**"))
                break


        if self.brv_flag == 1: # Constant Volume Batch Reactor

            str_0 = ("Enter batch reactor volume in SI units. (Enter 1 if reactor volume is unknown):")
            str_r = "Enter batch reactor volume in SI units, again (Enter 1 if reactor volume is unknown):"

            for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

                if i == self.amax:
                    raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

                if i == 0:
                    self.V0 = input(str_0)
                else:
                    self.V0 = input(str_r)

                print("Entered reactor volume is:", self.V0)

                try:
                    self.V0 = ast.literal_eval(self.V0) # Evaluate user input
                except Exception as emsg: # Invalid python expression
                    print(emsg)
                    continue

                if not (isinstance(self.V0,int) or isinstance(self.V0,float)):
                    print("Entered object is not a number.")
                    continue

                # Check is unnecessary as along as ast.literal_eval is used
                # np.nan is an invalid python expression
                # "np.nan" converts it to string data type
                if np.isnan(self.V0) == True:
                    print("A NaN value has been entered. NaN value is not allowed.")
                    continue

                if self.V0 <= 0:
                    print("Initial volume is strictly positive. Entered value is either zero or negative.")
                    continue

                # Here, all checks have been satisfied.
                if i <= (self.amax-1):
                    self.n0 = self.c_vals[0,:]*self.V0 # Initial number of moles
                    display(Markdown("**Data for initial volume received successfully.**"))
                    break



        elif self.brv_flag == 2: # Variable Volume Batch Reactor



            self.vol_data_struct_disp()
            self.add_vol_data() # Get volume data from csv file

            self.V0 = self.V_vals[0] # Initial reactor volume
            self.n0 = self.c_vals[0,:]*self.V0 # Initial number of moles








    def add_molecular_weight(self):

        str_0a = (f"Enter the molecular weights of {self.S} participating species as a one-dimensional list with {self.S} elements. "
                "The order of species in the list should be the same as the order in which they appear in the columns of "
                "the stoichiometric matrix. Example of one dimensional list with molecular weights of Nitrogen and Hydrogen "
                "(correct to three decimal places): [14.007, 2.016].")
        str_0b = f"Enter Molecular Weight Data of {self.S} participating species:"
        str_r = f"Enter Molecular Weight Data of {self.S} participating species again:"

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                self.Mw = input(str_0b)
            else:
                self.Mw = input(str_r)

            print(f"Entered molecular weights of {self.S} participating species is:", self.Mw)

            try:
                self.Mw = ast.literal_eval(self.Mw) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.Mw,list): # N is not a list
                print("Entered object is not a list.")
                continue

            try:
                self.Mw = np.array(self.Mw) # Convert N to numpy array
            except Exception as emsg: # List Mw could not be converted to numpy array
                print("Unable to convert input to numpy array, possibly due to inhomogeneous shape i.e. unequal number of columns across rows.")
                print("Further information:")
                print(emsg)
                continue

            if not np.issubdtype(self.Mw.dtype, np.number):
                print("Molecular weights must be numbers.")
                continue

            if not np.all(np.isreal(self.Mw)):
                print("Some molecular weights are complex numbers.")
                continue


            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan as an element of Mw is an invalid python expression
            # "np.nan" as an element of Mw converts it to string data type
            # [1,,] is an invalid python expression
            if np.any(np.isnan(self.Mw)):
                print("At least one element in the entered list is a NaN value. NaN values are not allowed.")
                continue


            if self.Mw.ndim != 1:
                print("Number of dimensions of entered list is not equal to 1, possibly because elements of entered list are not scalars.")
                continue

            if len(self.Mw) != self.S:
                print(f"Number of elements in the list is {len(self.Mw)}, whereas the number of participating species is {self.S}. The two should be equal.")
                continue

            if not (np.all(self.Mw > 0)) == True:
                print("Molecular weight cannot be negative or zero. At least one molecular weight entry is either negative or zero.")
                continue

            if not np.all(np.abs(np.dot(self.N,self.Mw.reshape(-1,1))) <= 10**(-10)): # Ideally, np.dot(N,Mw) = Null vector. Checking if each element is <= 10**(-10) allows small inaccuracy in molecular weight data.
                print("Mass conservation is being violated in one or more reactions. Molecular weights and stoichiometric matrix are inconsistent.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for molecular weights received successfully.**"))
                break








    def flow_data_struct_disp(self):

        display(Markdown("**• The following is pertinent information regarding the structure of the flow rate data file.**"))
        print("• Column headers are not allowed.")
        str_1 = ("• Column 1 in the file must contain time points at which the flow rate data is "
                 "collected. The next P columns i.e. columns 2 to (P + 1) should contain flow rate data of "
                 "P inlet streams. If an outlet stream is present (reactor configuration is CSTR), then the "
                 "last column i.e. column number (P + 2) should contain flow rate data of the outlet "
                 "stream.")
        print(textwrap.fill(str_1,width=125))
        print("• The time points should be the same as those in the concentration data file.")
        str_2 = ("• If the flow rate is constant for a particular stream, then all entries in the "
                  "corresponding column should have the same value.")
        print(textwrap.fill(str_2,width=125))
        print("• The file should be a csv file.")
        print("• All data should be in SI units.")





    def add_flowrate_data_from_file(self):

        str_0a = "Upload flow rate data file to local storage and enter flow rate data file name."
        str_0b = "Enter flow rate data file name:"
        str_r = "Enter flow rate data file name again:"
        str_max = "Maximum number of attempts exceeded. Please start again."

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                fname = input(str_0b)
            else:
                fname = input(str_r)

            print("Entered file name is:", fname)

            try:
                self.time_flow_df = pd.read_csv(fname,header=None) # Read file
            except Exception as emsg: # File not uploaded or some other error
                print(emsg)
                continue

            try:
                self.time_flow = np.array(self.time_flow_df) # Read file
            except Exception as emsg: # Dataframe could not be converted to numpy array
                print(emsg)
                continue

            if not np.issubdtype(self.time_flow.dtype, np.number):
                print("Time and flow rate values must be numbers. Non-numeric data is present in flow rate data file.")
                continue

            if not np.all(self.time_flow >=0):
                print("All values must be non-negative. At least one value in the flow rate data file is negative.")
                continue

            if not int(self.time_flow[0][0]) == 0:
                print("First time point must be zero. It is not equal to zero in the flow rate data file.")
                continue

            if not np.all(np.diff(self.time_flow[:,0]) > 0):
                print("Time points must be strictly monotonously increasing. This is not the case in the flow rate data file.")
                continue

            # Check for missing data

            if np.any(np.isnan(self.time_flow)):
                print("At least one value in the flow rate data file is missing. Missing values are not allowed.")
                continue


            if not np.all(self.time_flow[:,0] == self.time_conc[:,0]) == True:
                print("Time data points in concentration data file are not the same as those in the flow rate data file.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Time and flow rate data received successfully.**"))
                if self.rc_flag == 2:
                    self.P = self.time_flow.shape[1] - 1 # first column is time
                    self.uin_vals = self.time_flow[:,1:]
                    if self.P == 1:
                        self.uin_vals = self.uin_vals.reshape(-1,1)
                    self.uout_vals = np.zeros(np.shape(self.time_flow)[0]).reshape(-1,1)
                    display(Markdown(f"**The number of inlets is {self.P} and the number of outlets is 0.**"))
                elif self.rc_flag == 3:
                    self.P = self.time_flow.shape[1] - 2 # first column is time, last column is outlet
                    self.uin_vals = self.time_flow[:,1:-1]
                    if self.P == 1:
                        self.uin_vals = self.uin_vals.reshape(-1,1)
                    self.uout_vals = self.time_flow[:,-1].reshape(-1,1)
                    display(Markdown(f"**The number of inlets is {self.P} and the number of outlets is 1.**"))
                break




    def add_winhat(self):

        str_0a = ("Enter Winhat as a two-dimensional list. Example of Winhat for a reaction system with 3 participating species "
                "(A,B,C) in a reactor with 2 inlets: [[0, 0.5], [0.25, 0.5], [0.75, 0]]. In inlet 1 (first column), the mass "
                "fractions of species A, B and C are 0, 0.25 and 0.75, respectively. In inlet 2, the mass fractions of "
                "species A, B and C are 0.5, 0.5 and 0, respectively.")
        str_0b = "Enter Winhat matrix:"
        str_r = "Enter Winhat data matrix:"

        for i in range(self.amax+1): # i = 0,1,...,(amax-1),amax => attempts = 1,2,...,amax,(amax+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                self.Winhat = input(str_0b)
            else:
                self.Winhat = input(str_r)

            print("Entered Winhat matrix is:", self.Winhat)

            try:
                self.Winhat = ast.literal_eval(self.Winhat) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.Winhat,list): # Winhat is not a list
                print("Entered object is not a list.")
                continue

            try:
                self.Winhat = np.array(self.Winhat) # Convert Winhat to numpy array
            except Exception as emsg: # List Winhat could not be converted to numpy array
                print("Unable to convert input to numpy array, possibly due to inhomogeneous shape i.e. unequal number of columns across rows.")
                print("Further information:")
                print(emsg)
                continue

            if not (np.issubdtype(self.Winhat.dtype,np.number)):
                print("Elements of stoichiometric matrix must be numbers.")
                continue

            if not np.all(np.isreal(self.Winhat)):
                print("Some elements of stoichiometric matrix are complex.")
                continue

            if self.Winhat.ndim != 2:
                print("Number of dimensions of stoichiometric matrix is not equal to 2.")
                continue

            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan as an element of Winhat is an invalid python expression
            # "np.nan" as an element of Winhat converts it to string data type
            # [[1,,]] is an invalid python expression
            if np.any(np.isnan(self.Winhat)) == True:
                print("At least one element in Winhat is a NaN value. NaN values are not allowed.")
                continue

            if not ( (np.all(self.Winhat >= 0) == True) and (np.all(self.Winhat <= 1) == True) ):
                print("Mass fractions must lie between 0 and 1. At least one entry is outside the permissible range.")
                continue

            if not np.all(np.sum(self.Winhat,axis=0)==1) == True:
                print("Sum of mass fractions for each inlet stream must equal 1. This is not the case in the entered data.")
                continue

            if self.Winhat.shape[0] != self.S:
                print("The number of species (rows) in Winhat is not equal to the number of species (columns) of the stoichiometric matrix.")
                continue

            if self.Winhat.shape[1] != self.P:
                print("The number of inlets (columns) in Winhat is not equal to the number of inlets in the flow rate data file.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for Winhat received successfully.**"))
                break





    def add_solvent_data(self):

        str_0a = "Does the reaction system have a solvent that is not a participant in any of the reactions ? Enter \"y\" for yes and \"n\" for no."
        str_0b = "Enter solvent information:"
        str_r = "Enter solvent information again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(str_0a)
                self.sol_flag = input(str_0b)
            else:
                self.sol_flag = input(str_r)

            print("Entered data is:", self.sol_flag)

            try:
                self.sol_flag = ast.literal_eval(self.sol_flag) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print(emsg)
                continue

            if not isinstance(self.sol_flag,str): # sol_flag is not a string
                print("Entered object is not a string.")
                continue

            if isinstance(self.sol_flag,str) and self.sol_flag != "y" and self.sol_flag != "n":
                print("Invalid entry. Allowed values are: \"y\" = yes; \"n\" = no")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Solvent information received successfully.**"))
                break

        if self.sol_flag == "y":

            str_0 = "Enter solvent molecular weight:"
            str_r = "Enter solvent molecular weight again:"

            for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

                if i == self.amax:
                    raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

                if i == 0:
                    self.M_ws = input(str_0)
                else:
                    self.M_ws = input(str_r)

                print("Entered solvent molecular weight is:", self.M_ws)

                try:
                    self.M_ws = ast.literal_eval(self.M_ws) # Evaluate user input
                except Exception as emsg: # Invalid python expression
                    print(emsg)
                    continue

                if not (isinstance(self.M_ws,int) or isinstance(self.M_ws,float)):
                    print("Entered object is not a number.")
                    continue

                # Check is unnecessary as along as ast.literal_eval is used
                # np.nan is an invalid python expression
                # "np.nan" converts it to string data type
                if np.isnan(self.M_ws) == True:
                    print("A NaN value has been entered. NaN value is not allowed.")
                    continue

                if self.M_ws <= 0:
                    print("Molecular weight is strictly positive. Entered value is either zero or negative.")
                    continue

                # Here, all checks have been satisfied.
                if i <= (self.amax-1):
                    display(Markdown("**Data for solvent molecular weight received successfully.**"))
                    break

            str_0 = "Enter initial number of moles of solvent:"
            str_r = "Enter initial number of moles of solvent again:"

            for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

                if i == self.amax:
                    raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

                if i == 0:
                    self.n_s0 = input(str_0)
                else:
                    self.n_s0 = input(str_r)

                print("Entered initial number of moles of solvent is:", self.n_s0)

                try:
                    self.n_s0 = ast.literal_eval(self.n_s0) # Evaluate user input
                except Exception as emsg: # Invalid python expression
                    print(emsg)
                    continue

                if not (isinstance(self.n_s0,int) or isinstance(self.n_s0,float)):
                    print("Entered object is not a number.")
                    continue

                # Check is unnecessary as along as ast.literal_eval is used
                # np.nan is an invalid python expression
                # "np.nan" converts it to string data type
                if np.isnan(self.n_s0) == True:
                    print("A NaN value has been entered. NaN value is not allowed.")
                    continue

                if self.n_s0 <= 0:
                    print("Initial number of moles of solvent is strictly positive. Entered value is either zero or negative.")
                    continue

                # Here, all checks have been satisfied.
                if i <= (self.amax-1):
                    display(Markdown("**Data for initial number of moles of solvent received successfully.**"))
                    break





    def add_n_init(self):

        str_0a = (f"Enter the initial number of moles of {self.S} participating species as a one-dimensional "
                f"list with {self.S} elements. The order of species in the list should be the same as the "
                "order in which they appear in the columns of the stoichiometric matrix. Example of "
                "one dimensional list with initial number of moles of two species A, B is [0,1] "
                "where the initial number of moles of species A is zero and that of species B is 1.")
        str_0b = f"Enter initial number of moles of {self.S} participating species:"
        str_r = f"Enter initial number of moles of {self.S} participating species again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                self.n0 = input(str_0b)
            else:
                self.n0 = input(str_r)

            print(f"Entered molecular weights of {self.S} participating species is:", self.n0)

            try:
                self.n0 = ast.literal_eval(self.n0) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print("Entered object is not a valid python expression. Further information:")
                print(emsg)
                continue

            if not isinstance(self.n0,list): # n0 is not a list
                print("Entered object is not a list.")
                continue

            try:
                self.n0 = np.array(self.n0) # Convert n0 to numpy array
            except Exception as emsg: # List n0 could not be converted to numpy array
                print("Unable to convert input to numpy array, possibly due to inhomogeneous shape i.e. unequal number of columns across rows.")
                print("Further information:")
                print(emsg)
                continue

            if not np.issubdtype(self.n0.dtype, np.number):
                print("Moles must be numbers.")
                continue

            if not np.all(np.isreal(self.n0)):
                print("Some moles are complex numbers.")
                continue

            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan as an element of n0 is an invalid python expression
            # "np.nan" as an element of n0 converts it to string data type
            # [1,,] is an invalid python expression
            if np.any(np.isnan(self.n0)):
                print("At least one element in the entered list is a NaN value. NaN values are not allowed.")
                continue

            if self.n0.ndim != 1:
                print("Number of dimensions of entered list is not equal to 1, possibly because elements of entered list are not scalars.")
                continue

            if len(self.n0) != self.S:
                print(f"Number of elements in the list is {len(self.n0)}, whereas the number of participating species is {self.S}. The two should be equal.")
                continue

            if np.any(np.isnan(self.n0)) == True:
                print("At least one element in entered list is a NaN value. NaN values are not allowed.")
                continue

            if not (np.all(self.n0 >= 0)) == True:
                print("Moles cannot be negative. At least one entry is negative.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for initial number of moles received successfully.**"))
                break




    def add_init_vol(self):

        str_0 = "Enter initial volume of the reacting mixture (in SI units):"
        str_r = "Enter initial volume of the reacting mixture (in SI units) again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                self.V0 = input(str_0)
            else:
                self.V0 = input(str_r)

            print("Entered initial volume is:", self.V0)

            try:
                self.V0 = ast.literal_eval(self.V0) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print(emsg)
                continue

            if not (isinstance(self.V0,int) or isinstance(self.V0,float)):
                print("Entered object is not a number.")
                continue

            # Check is unnecessary as along as ast.literal_eval is used
            # np.nan is an invalid python expression
            # "np.nan" converts it to string data type
            if np.isnan(self.V0) == True:
                print("A NaN value has been entered. NaN value is not allowed.")
                continue

            if self.V0 <= 0:
                print("initial volume is strictly positive. Entered value is either zero or negative.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data for initial volume received successfully.**"))
                break





    def add_n0_V0_vol_flow_rct(self):

        # Add attributes n0, V0, rho, volume data (if provided by user)

        str_0a = ("Information is needed on either n0 (option A) or V0 (option B) "
                  "or time series reactor volume data (option C) where n0 is the initial number "
                  "of moles of participating species and V0 is the initial volume "
                  "of the reacting mixture. Note that a constant density "
                  "system is assumed when n0 or V0 is provided. Enter 1 for option A, "
                  "2 for option B, 3 for option C.")
        str_0b = "Enter requested information:"
        str_r = "Enter requested information again:"

        for i in range(self.amax+1): # i = 0,1,...,(n-1),n => attempts = 1,2,...,n,(n+1)

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a,width=120))
                self.flwrct_flag = input(str_0b)
            else:
                self.flwrct_flag = input(str_r)

            print("Entered information is:", self.flwrct_flag)

            try:
                self.flwrct_flag = ast.literal_eval(self.flwrct_flag) # Evaluate user input
            except Exception as emsg: # Invalid python expression
                print(emsg)
                continue

            if not isinstance(self.flwrct_flag,int): # self.rhov0_flag is not an integer
                print("Entered object is not an integer.")
                continue

            if isinstance(self.flwrct_flag,int) and (self.flwrct_flag != 1) and (self.flwrct_flag != 2) and (self.flwrct_flag != 3):
                print("Invalid value has been entered. Allowed values are: 1, 2 and 3.")
                continue

            # Here, all checks have been satisfied.
            if i <= (self.amax-1):
                display(Markdown("**Data received successfully.**"))
                break

        if self.flwrct_flag == 1: # user will provide n0

            self.add_n_init()

            V0_vals = self.n0/self.c_vals[0,:]
            V0_cov = (np.std(V0_vals,ddof=1)/np.mean(V0_vals))*100

            if V0_cov > 1:

                raise ValueError(f"Initial volume calculated using concentration data and data on number of moles  of participating species is varying with a coefficient of variation {V0_cov}. This is higher than the acceptable threshold of 5\%")

            else:

                self.V0 = np.mean(V0_vals)

                if self.sol_flag == "y":
                    self.m0 = np.sum(np.multiply(self.n0,self.Mw)) + self.M_ws*self.n_s0
                else:
                    self.m0 = np.sum(np.multiply(self.n0,self.Mw))

                self.rho = self.m0/self.V0

        elif self.flwrct_flag == 2: # user will provide V0

            self.add_init_vol()

            self.n0 = self.c_vals[0,:]*self.V0

            if self.sol_flag == "y":
                self.m0 = np.sum(np.multiply(self.n0,self.Mw)) + self.M_ws*self.n_s0
            else:
                self.m0 = np.sum(np.multiply(self.n0,self.Mw))

            self.rho = self.m0/self.V0

        elif self.flwrct_flag == 3: # user will provide volume data file

            self.vol_data_struct_disp()
            self.add_vol_data()

            self.V0 = self.V_vals[0]
            self.n0 = self.c_vals[0,:]*self.V0

            if self.sol_flag == "y":
                self.m0 = np.sum(np.multiply(self.n0,self.Mw)) + self.M_ws*self.n_s0
            else:
                self.m0 = np.sum(np.multiply(self.n0,self.Mw))

                self.rho = self.m0/self.V0


    def reactor_vol_evolve(self,V,t,*aparam):

        t_vals = aparam[0]
        uin_vals = aparam[1]
        out_vals = aparam[2]
        rho = aparam[3]


        f_uin = interpolate.interp1d(t_vals,uin_vals,axis=0,fill_value='extrapolate',kind='cubic')
        uin_t = f_uin(t).flatten().reshape(-1,1)

        f_uout = interpolate.interp1d(t_vals,out_vals,axis=0,fill_value='extrapolate',kind='cubic')
        uout_t = f_uout(t).flatten().item() # item() extracts float value

        dVdt = (np.sum(uin_t.flatten()) - uout_t )/rho

        return dVdt


    def compute_n_vals(self):

        if self.rc_flag == 1: # Batch reactor

            if self.brv_flag == 1: # constant volume batch reactor

                self.n_vals = (self.c_vals)*self.V0

            elif self.brv_flag == 2: # variable volume batch reactor (data provided in volume data file)

                V_mat = np.tile(self.V_vals.reshape(-1,1),(1,self.c_vals.shape[1]))

                self.n_vals = np.multiply(self.c_vals,V_mat)

        else: # Flow reactor

            if self.flwrct_flag == 1 or self.flwrct_flag == 2: # n0 or V0 has been provided by user

                aparam=(self.t_vals,self.uin_vals,self.uout_vals,self.rho)

                V_sol = odeint(self.reactor_vol_evolve,self.V0,self.t_vals,args=aparam)

                V_sol = np.array(V_sol).reshape(-1,1)

                V_mat = np.tile(V_sol,(1,self.c_vals.shape[1]))

                self.n_vals = np.multiply(self.c_vals,V_mat)

            elif self.flwrct_flag == 3: # Volume data has been provided by user in volume data file

                V_mat = np.tile(self.V_vals.reshape(-1,1),(1,self.c_vals.shape[1]))

                self.n_vals = np.multiply(self.c_vals,V_mat)




    # wanted to use something like default dict but in a callable way
    # instead of subscriptable, hence this hack
    def default_dict_v(self, x):
        """Returns an array with self.v_value with length of x.

        Args:
            x: A value or list-like object.

        Returns:
            numpy.ndarray or scalar: If x is list-like, returns a 1D numpy array of length len(x)
                with all elements set to self.v_value. If x is not list-like, returns self.v_value.
        """
        if pd.api.types.is_list_like(x):
            return np.array([self.v_value] * len(x))
        return self.v_value

    def default_dict_uin(self, x):
        """Returns a broadcasted numpy array of shape (len(x),) + self.uin_value.shape
        if x is list-like, otherwise returns self.uin_value.

        Args:
            x: A value or list-like object.

        Returns:
            numpy.ndarray or scalar: If x is list-like, returns a broadcasted numpy array of shape
                (len(x),) + self.uin_value.shape with all elements set to self.uin_value.
                If x is not list-like, returns self.uin_value.
        """
        if pd.api.types.is_list_like(x):
            return np.broadcast_to(self.uin_value, (len(x),) + self.uin_value.shape)
        return self.uin_value

    def default_dict_uout(self, x):
        """Returns an array with self.uout_value with length of x.

        Args:
            x: A value or list-like object.

        Returns:
            numpy.ndarray or scalar: If x is list-like, returns a 1D numpy array of length len(x)
                with all elements set to self.uout_value.
                If x is not list-like, returns self.uout_value.
        """
        if pd.api.types.is_list_like(x):
            return np.array([self.uout_value] * len(x))
        return self.uout_value

    def add_stoichiometry_data(self, N):
        """Adds stoichiometry data to the class instance and updates related attributes.

        Args:
            N: A list-like or numpy array representing the stoichiometry matrix.

        Note:
            - If N is a list, it will be converted to a numpy array.
            - The shape of the stoichiometry matrix should be (R, S),
            where R represents the number of reactions
            and S represents the number of species.
        """
        if isinstance(N, list):
            N = np.array(N)
        self.N = N
        self.R = N.shape[0]
        self.S = N.shape[1]

    def add_molweight_data(self, Mw):
        """Adds molecular weight data to the class instance and updates the Mw attribute.

        Args:
            Mw: A list-like or numpy array representing the molecular weight matrix.

        Raises:
            ValueError: If the molecular weight matrix Mw is not a square matrix.
        """
        if isinstance(Mw, list):
            if isinstance(Mw[0], list):
                Mw = np.array(Mw)
            else:
                Mw = np.diag(Mw)

        if Mw.shape[0] != Mw.shape[1]:
            raise ValueError("Molecular weight matrix: Mw must be a square matrix")

        self.Mw = Mw

    def add_volume_data(self, V, kind="linear"):
        """Adds volume data to the class instance and updates the V attribute.

        Args:
            V: A value, dataframe, dictionary, or file location representing the volume data.
            kind (str, optional): The interpolation method for extending the data.
                Defaults to "linear".

        Raises:
            ValueError: If the volume data has an incorrect format.
        """
        if isinstance(V, (int, float)):
            self.v_value = V
            self.V = self.default_dict_v

        else:
            if isinstance(V, pd.DataFrame):
                time = V.index
                y = np.array(V.iloc[:, 0])

            elif isinstance(V, dict):
                time = list(V.keys())
                y = list(V.values())

            elif isinstance(V, str):
                V = read_file(V)
                time = V.index
                y = np.array(V.iloc[:, 0])

            else:
                raise ValueError(
                    "Incorrect format for volume data. Supported format: \
                        (int, float), dataframe, dict or file location"
                )

            # extending the data linearly using the last two data points.
            # Since It is normal for the odeint solver to evaluate your function
            # at time values past the last requested time. Most ODE solvers work
            # this way--they take internal time steps with sizes determined by their
            # error control algorithm, and then use their own interpolation
            # to evaluate the solution at the times requested by the user.

            dt = max(3, (time[-1] - time[0]) * 5)
            # Slope of the last segment.
            m = (y[-1] - y[-2]) / (time[-1] - time[-2])
            # Extended final time.
            time_ext = time[-1] + dt
            # Extended final data value.
            y_ext = y[-1] + m * dt
            # Extended arrays.
            time = np.append(time, time_ext)
            y = np.append(y, y_ext)

            if kind == "cubic_smooth":  # TEST CAREFULLY
                self.V, _ = cubic_smooth(
                    time, y, smooth=0.98, refit=False
                )  # _ is first derivative

            else:  # Returns (t,p). Will have to fix.
                self.V = sc.interpolate.interp1d(time, y, kind=kind)

    def add_Winhat_data(self, Winhat):
        """Adds Winhat data to the class instance and updates the Winhat attribute.

        Args:
            Winhat: A list-like or numpy array representing the Winhat data.

        """
        if isinstance(Winhat, list):
            Winhat = np.array(Winhat)
        self.Winhat = Winhat

        if len(Winhat.shape) == 1:
            self.P = 1

        else:
            self.P = Winhat.shape[1]

    def add_uin_data(self, uin, kind="linear"):
        """Adds uin data to the class instance and updates the uin attribute.

        Args:
            uin: A value, list, numpy array, pandas DataFrame, or file location
                representing the uin data.
            kind (str, optional): The interpolation method for extending the data.
                Defaults to "linear".

        Raises:
            ValueError: If the uin data has an unsupported file format.
        """
        if isinstance(uin, (float, int)):
            uin = np.array([uin])
            self.uin_value = uin
            self.uin = self.default_dict_uin

        elif isinstance(uin, list):
            uin = np.array(uin)
            uin = np.squeeze(uin)
            self.uin_value = uin
            self.uin = self.default_dict_uin

        elif isinstance(uin, np.ndarray):
            uin = np.squeeze(uin)
            self.uin_value = uin
            self.uin = self.default_dict_uin

        elif isinstance(uin, (pd.DataFrame, str)):
            if isinstance(uin, str):
                uin = read_file(uin)

            time = uin.index
            values = uin.values

            dt = max(3, (time[-1] - time[0]) * 4)
            # Slope of the last segment.
            m = (values[-1] - values[-2]) / (time[-1] - time[-2])
            # Extended final time.
            time_ext = time[-1] + dt
            # Extended final data value.
            values_ext = values[-1] + m * dt
            # Extended arrays.
            time = np.append(time, time_ext)
            values = np.vstack([values, values_ext])

            if kind == "cubic_smooth":  # TEST CAREFULLY
                self.uin, _ = cubic_smooth(time, values, smooth=0.98, refit=False)
            else:
                self.uin = sc.interpolate.interp1d(time, values, kind=kind, axis=0)

        else:
            raise ValueError(
                "Unsupported file format. uin should be either int, float, list, \
                    numpy array (p,1) shape, pandas dataframe or file directory."
            )

    def add_uout_data(self, uout, kind="linear"):
        """Adds uout data to the class instance and updates the uout attribute.

        Args:
            uout: A value, pandas DataFrame, or file location representing the uout data.
            kind (str, optional): The interpolation method for extending the
                data. Defaults to "linear".

        Raises:
            ValueError: If the uout data has an unsupported file format.
        """
        if isinstance(uout, (float, int)):
            self.uout_value = np.array([uout])
            self.uout = self.default_dict_uout

        elif isinstance(uout, (pd.DataFrame, str)):
            if isinstance(uout, str):
                uout = read_file(uout)

            time = uout.index
            values = uout.values

            dt = max(3, (time[-1] - time[0]) * 4)
            # Slope of the last segment.
            m = (values[-1] - values[-2]) / (time[-1] - time[-2])
            # Extended final time.
            time_ext = time[-1] + dt
            # Extended final data value.
            values_ext = values[-1] + m * dt
            # Extended arrays.
            time = np.append(time, time_ext)
            values = np.vstack([values, values_ext])

            if kind == "cubic_smooth":  # TEST CAREFULLY
                self.uout, _ = cubic_smooth(time, values, smooth=0.98, refit=False)
            else:
                self.uout = sc.interpolate.interp1d(time, values, kind=kind, axis=0)

        else:
            raise ValueError(
                "Unsupported file format. uout should be either int, float, pandas \
                    dataframe or file directory."
            )

    def add_n0_data(self, n0):
        """Adds n0 data to the class instance and updates the n0 attribute.

        Args:
            n0: A list-like or numpy array representing the n0 data.
        """
        if isinstance(n0, list):
            n0 = np.array(n0)
        self.n0 = n0

    def add_reactor_config(self, V, Winhat, uin, uout, config):
        """Adds reactor configuration data to the class instance.

        Args:
            V: The volume of the reactor.
            Winhat: The Winhat data.
            uin: The uin data.
            uout: The uout data.
            config (str): The configuration type of the reactor. Supported values:
                "batch", "semi-batch", "cstr".

        Returns:
            tuple: A tuple containing the updated values for V, Winhat, uin, and uout.

        Raises:
            ValueError: If the configuration is invalid or if required arguments
                are not provided for a specific configuration.
        """
        if config is None and ((Winhat is None) or (uin is None) or (uout is None)):
            raise ValueError(
                "If config is not specified, other arguments should be provided"
            )

        if config == "batch":
            if Winhat is not None:
                warnings.warn(
                    f"Winhat is provided for config = {config}. Using Winhat = {[0]*self.S}",
                    stacklevel=2,
                )
            if uin is not None:
                if uin == 0:
                    pass
                else:
                    raise ValueError(
                        f"uin should not be specified for config = {config}. \
                            Non-zero uin provided."
                    )
            if uout is not None:
                if uout == 0:
                    pass
                else:
                    raise ValueError(
                        f"uout should not be specified for config = {config}. \
                            Non-zero uout provided."
                    )

            Winhat = [0] * self.S
            uin = 0
            uout = 0

        elif config == "semi-batch":
            if Winhat is None:
                raise ValueError(f"Winhat cannot be none/empty for config = {config}")
            if uin is None:
                raise ValueError(f"uin cannot be none/empty for config = {config}")
            if uout is not None:
                if uout == 0:
                    pass
                else:
                    raise ValueError(
                        f"uout should not be specified for config = {config}. \
                            Non-zero uout value provided."
                    )
            uout = 0

        elif config == "cstr":
            pass

        else:
            pass

        return V, Winhat, uin, uout


    def add_concentration_data_from_file(self):

        str_0a = "Upload concentration data file to local storage and enter concentration data file name."
        str_0b = "Enter concentration data file name:"
        str_r = "Enter concentration data file name again:"
        str_max = "Maximum number of attempts exceeded. Please start again."

        for i in range(self.amax+1):

            if i == self.amax:
                raise ValueError("Maximum number of attempts exceeded. Aborting operation. Please start again.")

            if i == 0:
                print(textwrap.fill(str_0a, width=120))
                fname = input(str_0b)
            else:
                fname = input(str_r)

            print("Entered file name is:", fname)

            try:
                self.time_conc_df = pd.read_csv(fname)
            except Exception as emsg:
                print(emsg)
                continue

            try:
                self.time_conc = np.array(self.time_conc_df)
            except Exception as emsg:
                print(emsg)
                continue

            if not np.issubdtype(self.time_conc.dtype, np.number):
                print("Time and concentration values must be numbers. Non-numeric data detected.")
                continue

            # ==================================================================
            # >>> CHANGED: Improved missing-data handling (with reactor logic)
            # ==================================================================
            time_col = self.time_conc[:, 0]
            conc_cols = self.time_conc[:, 1:]

            if np.any(np.isnan(time_col)):
                print("Missing values in time column are not allowed.")
                continue

            missing_conc_present = np.any(np.isnan(conc_cols))
            if missing_conc_present:
                print("Warning: Missing concentration data detected. Checking if reconstruction is possible...")

                # Batch → no inlet terms required
                if self.rc_flag == 1:
                    print("Reactor configuration is Batch → Wina and n0a are not required for reconstruction.")

                # Semi-Batch / CSTR → require Wina & n0a
                else:
                    if self.Wina is None:
                        print("Missing concentration data cannot be accepted because Wina (inlet composition of available species) is not available.")
                        continue
                    if self.n0a is None:
                        print("Missing concentration data cannot be accepted because n0a (initial moles of available species) is not available.")
                        continue

                    # Construct Na only for rank validation
                    available_species_list = list(self.time_conc_df.columns[1:])
                    idx = [self.S_l.index(sp) for sp in available_species_list]
                    Na_tmp = self.N[:, idx]

                    if np.linalg.matrix_rank(Na_tmp) != self.R:
                        print("Missing concentration data cannot be accepted because rank(Na) != R. "
                              "The provided subset of species is insufficient to determine the reaction system.")
                        continue

                    print("Subset of available species satisfies rank(Na) = R. Missing concentrations will be handled later.")
            # ==================================================================
            # <<< CHANGED END
            # ==================================================================

            if not np.all(self.time_conc >= 0, where=~np.isnan(self.time_conc)):
                print("All reported concentration values must be non-negative.")
                continue

            if not int(self.time_conc[0][0]) == 0:
                print("First time point must be zero.")
                continue

            if not np.all(np.diff(self.time_conc[:, 0]) > 0):
                print("Time points must be strictly monotonically increasing.")
                continue

            self.S_c = set(self.time_conc_df.columns[1:])
            if not len(self.S_c) <= self.S:
                print(f"Number of species in concentration file ({len(self.S_c)}) exceeds expected number ({self.S}).")
                continue

            if not self.S_c.issubset(self.S_s):
                sp_nms_all = ", ".join(sorted(self.S_l))
                sp_nms_cdf = ", ".join(sorted(self.S_c))
                print(f"One or more species names in concentration data file are invalid. "
                      f"Allowed names: {sp_nms_all}. Provided: {sp_nms_cdf}")
                continue

            # Success
            if i <= (self.amax - 1):
                display(Markdown("**Time and concentration data received successfully.**"))
                self.t_vals = time_col
                self.c_vals = conc_cols   # may include NaN if missing values allowed
                break


    ''' older func
    def add_concentration_data(
        self,
        conc,
        time=None,
        kind="cubic_smooth",
        smooth=None,
        refit=False,
        preprocess=None,
        threshold=0,
        window_length=None,
        polyorder=3,
        deriv=0,
        delta=1.0,
        axis=0,
        mode="interp",
        cval=0.0,
    ):
        """Adds concentration data to the class instance.

        Args:
            - conc (Union[pd.DataFrame, np.ndarray, str]): The concentration data.
                It can be provided as a pandas DataFrame, numpy array, or a file directory (str).
            - time (Optional[np.ndarray]): The time data corresponding to
                the concentration measurements.
            - kind (str): The interpolation method for smoothing the concentration data.
                 Default is "cubic_smooth".
            - smooth (Optional[float]): The smoothing parameter for cubic smoothing.
                 Default is None.
            - refit (bool): Whether to refit the smoothing function after extending the data.
                 Default is False.
            - preprocess (Optional[str]): Preprocessing method for the concentration data.
                 Supported values: "baseline_shift","threshold_cutoff", "savgol_filter".
                 Default is None.
            - threshold (float): The threshold value for threshold_cutoff preprocessing.
                 Default is 0.
            - window_length (Optional[int]): The window length for savgol_filter
                preprocessing. Default is None.
            - polyorder (int): The polynomial order for savgol_filter
                preprocessing. Default is 3.
            - deriv (int): The derivative order for savgol_filter preprocessing. Default is 0.
            - delta (float): The spacing between the time points for savgol_filter
                preprocessing. Default is 1.0.
            - axis (int): The axis along which the savgol_filter is applied. Default is 0.
            - mode (str): The extrapolation mode for savgol_filter
                preprocessing. Default is "interp".
            - cval (float): The constant value used for extrapolation
                when mode is "constant". Default is 0.0.

        Raises:
            ValueError: If the concentration data is not provided or
            if the problem is non-identifiable with the concentration provided.
        """
        self.methodology = None
        if conc is None:
            raise ValueError(
                "Concentration data is not provided. Missing concentration data"
            )

        if time is None:
            if isinstance(conc, (pd.DataFrame, str)):
                if isinstance(conc, str):
                    conc = read_file(conc)

                time = np.array(conc.index)
                conc = conc.values

        self.time = time
        unavailable_idx = [idx for idx, c in enumerate(conc[0]) if (np.isnan(c))]
        available_idx = [idx for idx, c in enumerate(conc[0]) if (not np.isnan(c))]

        if unavailable_idx == []:
            if True in np.isnan(self.N):
                raise ValueError("Stoichiometric Matrix contains NaN Value(s)")
            if True in np.isnan(self.Mw):
                raise ValueError("Molecular weight matrix (Mw) contains NaN Value(s). ")
            if True in np.isnan(self.Winhat):
                raise ValueError("Winhat contains NaN values")
            if True in np.isnan(self.n0):
                warnings.warn("n0 contains NaN values.", stacklevel=2)
                self.methodology = (
                    "rate"  # if n0 contains nan, only rate based can be possible
                )

            n0_ = np.reshape(self.n0, (self.n0.shape[0], 1))
            mat = np.concatenate([self.N.T, self.Win, n0_], axis=1)

            if (np.linalg.matrix_rank(mat) == self.R + self.P + 1) and (
                self.methodology is None
            ):
                self.methodology = 1  # Here, it can be both rate based or extent based
            else:
                raise ValueError(
                    "Rank of [N.T, Win, n0_] != R + p + 1. Problem is not identifiable."
                )

        else:
            self.unavailable_idx = unavailable_idx
            self.available_idx = available_idx
            conc = conc[:, available_idx]

            if hasattr(self, "N"):
                self.Na = self.N[:, available_idx]
                self.Nu = self.N[:, unavailable_idx]

            if hasattr(self, "Win"):
                self.Wina = self.Win[available_idx, :]
                self.Winu = self.Win[unavailable_idx, :]

            if hasattr(self, "n0"):
                self.n0a = self.n0[available_idx]
                self.n0u = self.n0[unavailable_idx]

            if True in np.isnan(self.N):
                raise ValueError("Stoichiometric Matrix contains NaN Value(s)")
            if True in np.isnan(self.Mw):
                raise ValueError("Molecular weight matrix (Mw) contains NaN Value(s). ")
            if True in np.isnan(self.Winhat):
                raise ValueError("Winhat contains NaN values")
            if True in np.isnan(self.n0):
                raise ValueError("n0 contains NaN values.")

            if np.linalg.matrix_rank(self.Na) != self.R:
                raise ValueError(
                    f"Rank of Available stoichiometric matrix (Na) should be atleast rank {self.R}"
                )

            self.methodology = 2  # Here, it is only extent based

        if preprocess == "baseline_shift":
            conc = baseline_shift(conc)

        elif preprocess == "threshold_cutoff":
            conc = threshold_cutoff(conc, threshold=threshold)

        elif preprocess == "savgol_filter":
            if window_length is None:
                window_length = int(
                    time.shape[0] / 5
                )  # default value for window length is timesteps/5

            conc = savgol_filter(
                conc, window_length, polyorder, deriv, delta, axis, mode, cval
            )

        n = (conc.T * self.V(time)).T  # number of moles
        dt = max(3, (time[-1] - time[0]) * 4)
        # Slope of the last segment.
        m1 = (conc[-1] - conc[-2]) / (time[-1] - time[-2])
        m2 = (n[-1] - n[-2]) / (time[-1] - time[-2])
        # Extended final time.
        time_ext = time[-1] + dt
        # Extended final data value.
        conc_ext = conc[-1] + m1 * dt
        n_ext = n[-1] + m2 * dt
        # Extended arrays.
        time = np.append(time, time_ext)
        conc = np.vstack([conc, conc_ext])
        n = np.vstack([n, n_ext])

        if kind == "cubic_smooth":
            self.n, self.dndt = cubic_smooth(
                time, n, smooth=smooth, refit=refit
            )  # this is a function
            # self.c = ((self.n(time)).T / self.V(time)).T # smoothed conc values, this is np array
            self.c, self.dcdt = cubic_smooth(
                time, conc, smooth=smooth, refit=refit
            )  # this is a function
        else:
            self.n = sc.interpolate.interp1d(time, n, kind)  # this is a function
            # self.c = ((self.n(time)).T / self.V(time)).T # smoothed conc values, this is np array
            self.c = sc.interpolate.interp1d(time, conc, kind)  # this is a function


    '''


<>:1231: SyntaxWarning: invalid escape sequence '\%'
<>:1231: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipython-input-4110603378.py:1231: SyntaxWarning: invalid escape sequence '\%'
  raise ValueError(f"Initial volume calculated using concentration data and data on number of moles  of participating species is varying with a coefficient of variation {V0_cov}. This is higher than the acceptable threshold of 5\%")


#Extents

In [ ]:
import numpy as np
from scipy.integrate import ode


def compute_matrices(N, Win, n0):
    """
    Compute matrices S0, q0, M0, Q0T to compute physically interpretable extents

    Args:
        N (array-like): The stoichiometric matrix.
        Win (array-like): The input stoichiometric matrix.
        n0 (array-like): The initial concentration data.

    Returns:
        tuple: A tuple containing the matrices q0T, S0T, M0T, Q0T.

    Raises:
        ValueError: If the rank of the [N.T Win] matrix is not equal to numReactions + numInlet.

    """
    if len(n0.shape) == 1:
        n0 = np.reshape(n0, (n0.shape[0], 1))

    numReactions = N.shape[0]
    numSpecies = N.shape[1]
    numInlet = Win.shape[1]

    mat = np.concatenate([N.T, Win], axis=1)

    if np.linalg.matrix_rank(mat) == numReactions + numInlet:
        U1, S1, V1T = np.linalg.svd(mat)
        Q = U1[:, numReactions + numInlet:]
        mat2 = np.concatenate([N.T, Q], axis=1)
        U2, S2, V2T = np.linalg.svd(mat2)
        L = U2[:, numSpecies - numInlet:]
        M = L @ np.linalg.pinv(Win.T @ L)
        ST = np.linalg.pinv(N.T) @ (np.identity(numSpecies) - (Win @ M.T))

        q0T = (np.ones((numSpecies - numReactions - numInlet, 1)).T @ Q.T) / (
            np.ones((numSpecies - numReactions - numInlet, 1)).T @ Q.T @ n0
        )
        S0T = ST @ (np.identity(numSpecies) - n0 @ q0T)
        M0T = M.T @ (np.identity(numSpecies) - n0 @ q0T)
        Q0T = Q.T @ (np.identity(numSpecies) - n0 @ q0T)

    else:
        raise ValueError(
            "Rank Error. Rank of [N.T Win] matrix != numReactions + numInlet."
        )

    return q0T, S0T, M0T, Q0T


def extents_derivative_v2(t, y, uin, uout):
    """
    Derivative function for xin, lamda, and mass with respect to time

    Args:
        t (float): The current time.
        y (array-like): The current state vector.
        uin (function): The input concentration function.
        uout (function): The output concentration function.

    Returns:
        array-like: The derivative of the state vector with respect to time.

    """
    m, lamda = y[0], y[1]
    xin = np.array(y[2:])
    dm_dt = np.sum(uin(t)) - uout(t)[0]
    dl_dt = -(uout(t)[0] / m) * lamda
    dxin_dt = uin(t) - (uout(t)[0] / m) * xin
    dxin_dt = list(dxin_dt)
    dydt = [dm_dt, dl_dt]
    dydt.extend(dxin_dt)
    return dydt


def compute_inlet_extents(uin, uout, time, n0, Mw):
    """
    Returns xin, lamda, m
    Computes xin, lamda and m given uin, uout, time, n0, and Mw data

    Uses the algorithm proposed in V2 (Incremental identification. Nirav et al. paper)

    Args:
        uin (function): The input concentration function.
        uout (function): The output concentration function.
        time (array-like): The time points for the simulation.
        n0 (array-like): The initial concentration data.
        Mw (array-like): The molecular weight data.

    Returns:
        tuple: A tuple containing the vectors xin, lamda, and m.

    """
    p = uin(0).shape[0]
    m0 = np.sum(n0 @ Mw)
    t0 = 0.0
    y0 = [m0, 1]
    y0.extend([0] * p)

    solver = ode(extents_derivative_v2)
    solver.set_integrator("dop853")
    solver.set_f_params(uin, uout)
    solver.set_initial_value(y0, t0)

    N_ = time.shape[0]
    t1 = time[-1]

    sol = np.empty((N_, p + 2))
    sol[0] = y0

    k = 1
    while solver.successful() and solver.t < t1:
        solver.integrate(time[k])
        sol[k] = solver.y
        k += 1

    m = sol[:, 0]
    lamda = sol[:, 1]
    lamda = np.reshape(lamda, (lamda.shape[0], 1))
    xin = sol[:, 2:]
    return xin, lamda, m

# Simulate

In [ ]:
import numpy as np
from scipy.integrate import odeint
from functools import partial
# from extents import compute_matrices
# from datatools import DataTools
# from ratelawgen import RateLaw


class Simulate(DataTools):
    def __init__(
        self, N, Mw, V, Winhat=None, uin=None, uout=None, n0=None, config=None
    ):
        """
        Initializes a new instance of the Simulate class.

        Args:
            N (array-like): The stoichiometric list.
            Mw (array-like): The molecular weight data.
            V (array-like): The volume data.
            Winhat (array-like, optional): The input stoichiometric matrix. Defaults to None.
            uin (array-like, optional): The input concentration data. Defaults to None.
            uout (array-like, optional): The output concentration data. Defaults to None.
            n0 (array-like, optional): The initial concentration data. Defaults to None.
            config (dict, optional): The reactor configuration data. Defaults to None.

        Raises:
            ValueError: If n0 is None.

        """
        if n0 is None:
            raise ValueError("n0 cannot be None")

        self.add_stoichiometry_data(N)

        self.add_molweight_data(Mw)
        if self.Mw.shape[0] != self.S:
            raise ValueError(
                f"Dimension of Mw {self.Mw.shape} should be {self.S}x{self.S} and \
                    is not consistent with Stoichiometric Matrix N {self.N.shape}"
            )

        V, Winhat, uin, uout = self.add_reactor_config(V, Winhat, uin, uout, config)
        self.add_volume_data(V, kind="linear")

        self.add_Winhat_data(Winhat)
        if self.Winhat.shape[0] != self.S:
            raise ValueError(
                f"Dimension of Winhat {self.Winhat.shape} should be {self.S}x{self.P} \
                    and is not consistent with Stoichiometric Matrix N {self.N.shape}"
            )

        self.add_uin_data(uin, kind="linear")
        if self.uin(0).shape[0] != self.P:
            raise ValueError(
                f"Dimension of uin {self.uin.shape} is not consistent with Winhat {self.Win.shape}"
            )

        self.add_uout_data(uout, kind="linear")
        self.add_n0_data(n0)
        if self.n0.shape[0] != self.S:
            raise ValueError(
                f"Required {self.S} species for n0. Received {self.n0.shape[0]} species instead"
            )

        self.Win = np.linalg.pinv(self.Mw) @ self.Winhat  # (SxP) matrix
        if len(self.Win.shape) == 1:
            self.Win = np.reshape(self.Win, (self.Win.shape[0], 1))
        self.m0 = np.sum(self.Mw @ self.n0)

    def add_ratelaws(self, ratelaws, K):
        """
        Adds rate laws and associated rate constants.

        Args:
            ratelaws (list): The list of rate law functions.
            K (array-like): The rate constant values.

        """
        self.ratelaws = [
            partial(ratelaw.function, K=K[idx])
            if isinstance(ratelaw, RateLaw)
            else partial(ratelaw, K=K[idx])
            for idx, ratelaw in enumerate(ratelaws)
        ]

    def mole_balance(self, y, t):
        """
        Computes the mole balance equations.

        Args:
            y (array-like): The concentration vector.
            t (float): The current time.

        Returns:
            array-like: The derivative of the concentration vector.

        """
        c = y / self.V(t)
        rate = np.array([ratelaw(c) for ratelaw in self.ratelaws])
        m = np.sum(self.Mw @ y)  # current mass
        dydt = (
            (self.N.T @ rate * self.V(t))
            + (self.Win @ self.uin(t))
            - (self.uout(t) * y / m)
        )
        return dydt

    def run_simulation(self, time, alpha=0):
        """
        Runs the simulation.

        Args:
            time (array-like): The time points for the simulation.
            alpha (float, optional): The noise level for the simulation. Defaults to 0.

        Returns:
            dict: A dictionary containing the simulation results.

        """

        self.time = time
        sol = odeint(self.mole_balance, self.n0, time)

        nSamples, nSpecies = sol.shape[0], sol.shape[1]
        noise_mean = np.zeros(nSpecies)
        noise_std = (alpha / 100) * np.diag(np.ndarray.max(sol, axis=0))
        noise_cov = noise_std**2
        noise = np.random.multivariate_normal(
            mean=noise_mean, cov=noise_cov, size=nSamples
        )

        self._sol = sol + noise

        self._reaction_rate = np.array(
            [ratelaw(self._sol.T / self.V(time)) for ratelaw in self.ratelaws]
        )  # check this equation for vector of V

        self._flow_rate = ((self.N.T) @ self._reaction_rate * self.V(time)).T

        self._reaction_rate = self._reaction_rate.T

        q0T, S0T, M0T, Q0T = compute_matrices(self.N, self.Win, self.n0)

        self._xr = self._sol @ S0T.T

        self._xin = self._sol @ M0T.T

        self._lamda = self._sol @ q0T.T

        d = {
            "moles": self._sol,
            "reaction_rate": self._reaction_rate,
            "flow_rate": self._flow_rate,
            "xr": self._xr,
            "xin": self._xin,
            "xout": 1 - self._lamda,
        }

        return d

# Incremental

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import plotly.express as px
from functools import partial
from sklearn.metrics import mean_squared_error
# from ratelawgen import RateLaw  # custom package
# from datatools import DataTools
# from preprocess import num_params
import warnings
from joblib import Parallel, delayed
# from extents import compute_matrices, compute_inlet_extents
import heapq
import itertools
# from simulate import Simulate

# # from scipy.integrate import ode

class Incremental(Simulate, DataTools):
    def __init__(
        self, N, Mw, V, Winhat=None, uin=None, uout=None, n0=None, config=None
    ):
        """
        Initializes an Incremental object.

        Args:
            N: Stoichiometric matrix (RxS) where S is the number of species and
                R is the number of reactions.
            Mw: Molecular weight matrix (SxS) where S is the number of species.
            V: Volume data for the reactor.
            Winhat: Inlet flow rate matrix (SxP) where P is the number of inlets.
                Defaults to None.
            uin: Inlet concentration data. Defaults to None.
            uout: Outlet concentration data. Defaults to None.
            n0: Initial concentrations of species. Must be provided and cannot be None.
            config: Additional configuration data for the reactor. Defaults to None.

        Raises:
            ValueError: If n0 is None or if the dimensions of the provided data are inconsistent.

        """

        self.add_reactor_config_flag()

        self.add_number_reactions()

        self.add_number_species()

        self.add_stoichio()

        self.add_rxn_rev()

        self.conc_data_struct_disp()

        self.add_concentration_data_from_file()

        if self.rc_flag == 1:

            self.add_vol_batch()

        if (self.rc_flag == 2) or (self.rc_flag == 3):

            self.add_molecular_weight()

            self.flow_data_struct_disp()

            self.add_flowrate_data_from_file()

            self.add_winhat()

            self.add_solvent_data()

            self.add_n0_V0_vol_flow_rct()

        self.compute_n_vals()


        if n0 is None:
            raise ValueError("n0 cannot be None")

        self.add_stoichiometry_data(N)

        self.add_molweight_data(Mw)
        if self.Mw.shape[0] != self.S:
            raise ValueError(
                f"Dimension of Mw {self.Mw.shape} should be {self.S}x{self.S} \
                    and is not consistent with Stoichiometric Matrix N {self.N.shape}"
            )

        V, Winhat, uin, uout = self.add_reactor_config(V, Winhat, uin, uout, config)
        self.add_volume_data(V, kind="linear")

        self.add_Winhat_data(Winhat)
        if self.Winhat.shape[0] != self.S:
            raise ValueError(
                f"Dimension of Winhat {self.Winhat.shape} should be {self.S}x{self.P} \
                    and is not consistent with Stoichiometric Matrix N {self.N.shape}"
            )

        self.add_uin_data(uin, kind="linear")
        if self.uin(0).shape[0] != self.P:
            raise ValueError(
                f"Dimension of uin {self.uin.shape} is not consistent with \
                    Winhat {self.Win.shape}"
            )

        self.add_uout_data(uout, kind="linear")
        self.add_n0_data(n0)
        if self.n0.shape[0] != self.S:
            raise ValueError(
                f"Required {self.S} species for n0. Received {self.n0.shape[0]} species instead"
            )

        self.Win = np.linalg.pinv(self.Mw) @ self.Winhat  # (SxP) matrix
        if len(self.Win.shape) == 1:
            self.Win = np.reshape(self.Win, (self.Win.shape[0], 1))
        self.m0 = np.sum(self.Mw @ self.n0)

    def reaction_flow_f1(self):
        """
        Computes the reaction flow rate based on the concentration data.

        Returns:
            The computed reaction flow rate.

        Raises:
            ValueError: If concentration data is not specified. Add concentration data
                through the `add_concentration_data` method.
        """
        if not hasattr(self, "n"):
            raise ValueError(
                "Concentration data not specified. Add concentration data through\
                      add_concentration_data method"
            )

        mass = np.sum(self.n(self.time) @ self.Mw, axis=1)
        flow_rate = (
            self.dndt(self.time)
            - (self.uin(self.time) @ self.Win.T)
            + ((self.n(self.time) * ((self.uout(self.time).T / mass).T)))
        )
        return flow_rate

    def reaction_rate_r1(self):
        """
        Computes the reaction rate based on the reaction flow rate and reactor volume.

        Returns:
            The computed reaction rate.
        """
        flow_rate = self.reaction_flow_f1()
        reaction_rate = np.linalg.pinv(self.N.T) @ flow_rate.T * 1 / self.V(self.time)
        reaction_rate = reaction_rate.T
        return reaction_rate

    def rb_objective_function(self, K, reaction_rate, ratelaw, conc_dataT):
        """
        Computes the objective function for parameter estimation.

        Args:
            K: Parameter values for the rate law.
            reaction_rate: Computed reaction rates.
            ratelaw: Rate law function.
            conc_dataT: Transposed concentration data.

        Returns:
            The value of the objective function.

        Notes:
            This function is used in parameter estimation to determine the goodness of
                fit between the computed reaction rates and the rate law.

        """
        return np.sqrt(mean_squared_error(reaction_rate, ratelaw(conc_dataT, K)))

    def rb_est_params(self, reaction_rate, candidates: list):
        """
        Estimates Parameters given candidate rate laws and computed rates.

        Args:
            reaction_rate: Computed reaction rates.
            candidates: A list of candidate rate laws or RateLaw objects.

        Returns:
            Results from the optimization process for each candidate rate law.

        """
        # check for convergence. If it does not converge, reinitialize with a
        # different guessn(TO DO)

        results = []
        for idx, candidate in enumerate(candidates):
            if isinstance(candidate, RateLaw):
                print(f"\tProcessing expression {candidate.expression}")
                ratelaw = candidate.function
            else:
                print(f"\tProcessing {candidate.__str__()[10:-19]} Law....")
                ratelaw = candidate

            conc = self.c(self.time).T
            n_param = num_params(ratelaw, conc)
            x0 = np.random.rand(n_param)
            bnds = []
            for i in range(n_param):
                bnds.append((0, None))
            optim = sc.optimize.minimize(
                self.rb_objective_function,
                x0,
                args=(reaction_rate, ratelaw, conc),
                bounds=bnds,
            )
            results.append(optim)

        return results

    def rb_ci_parallelise(self, reaction_rate, conc, ratelaw, n_param):
        """
        Runs the parameter estimation process in parallel for bootstrapping.

        Args:
            reaction_rate: Computed reaction rates.
            conc: Concentration data.
            ratelaw: Rate law function.
            n_param: Number of parameters in the rate law.

        Returns:
            The optimized parameter values.

        """
        idx = np.random.choice(self.time.shape[0], self.time.shape[0], replace=True)
        bs_mdata = conc[idx, :]
        bs_compRRate = reaction_rate[idx]
        x0 = np.random.rand(n_param)
        bnds = []
        for i in range(n_param):
            bnds.append((0, None))
        optim = sc.optimize.minimize(
            self.rb_objective_function,
            x0,
            args=(bs_compRRate, ratelaw, bs_mdata.T),
            bounds=bnds,
        ).x
        return optim

    def conf_int_rate_based(
        self, reaction_rate, candidate, confidence, bootstraps, n_jobs
    ):
        """
        Computes the confidence interval for the rate-based parameter estimation.

        Args:
            reaction_rate: Computed reaction rates.
            candidate: Candidate rate law or RateLaw object.
            confidence: Confidence level for the interval.
            bootstraps: Number of bootstrap iterations.
            n_jobs: Number of parallel jobs.

        Returns:
            The confidence interval for the parameter estimation.

        """
        if isinstance(candidate, RateLaw):
            ratelaw = candidate.function
        else:
            ratelaw = candidate

        conc = self.c(self.time)
        n_param = num_params(ratelaw, conc.T)

        optims = []
        optims = Parallel(n_jobs=n_jobs)(
            delayed(self.rb_ci_parallelise)(reaction_rate, conc, ratelaw, n_param)
            for i in range(bootstraps)
        )

        CI = np.percentile(
            optims, [100 * (1 - confidence) / 2, 100 * (1 - (1 - confidence) / 2)]
        )
        return CI

    def eb_extent_rate_derivative(self, x, t, K, ratelaw):
        """
        Computes the derivative of computed extents.

        Args:
            x: Extent value.
            t: Time.
            K: Parameter values for the rate law.
            ratelaw: Rate law function.

        Returns:
            The derivative of the computed extents.

        Notes:
            This function is used to numerically find the extent value for a
                particular rate law by solving the derivative equation.

        """
        mass = np.sum(self.n(t) @ self.Mw)
        dxdt = ratelaw(self.c(t), K) * self.V(t) - (self.uout(t)[0] / mass) * x
        return dxdt

    def eb_objective_function(self, K, reaction_extent, ratelaw, time):
        """
        Computes the objective function for parameter estimation based on extent-based modeling.

        Args:
            K: Parameter values for the rate law.
            reaction_extent: Experimental reaction extents.
            ratelaw: Rate law function.
            time: Time values.

        Returns:
            The value of the objective function.

        Notes:
            This function compares the experimental reaction extents with the
            computed extents based on the rate law and parameters.
            It uses the root mean squared error (RMSE) as the measure of
            dissimilarity between the experimental and computed extents.

        """
        computed_extent = odeint(
            self.eb_extent_rate_derivative, 0, time, args=(K, ratelaw)
        )
        computed_extent = np.squeeze(computed_extent)
        loss = np.sqrt(mean_squared_error(reaction_extent, computed_extent))
        return loss

    def eb_est_params(self, reaction_extent, candidates: list, solver):
        """
        Estimates the parameters using extent-based modeling and multiple candidate rate laws.

        Args:
            reaction_extent: Experimental reaction extents.
            candidates: A list of candidate rate laws or rate law functions.
            solver: Optimization solver method.

        Returns:
            A list of optimization results for each candidate rate law.

        Notes:
            This function performs parameter estimation by minimizing the objective
            function based on extent-based modeling. It iterates over the candidates and
            finds the optimal parameter values using the specified optimization solver.

        """
        results = []
        for idx, candidate in enumerate(candidates):
            if isinstance(candidate, RateLaw):
                print(f"\tProcessing expression {candidate.expression}")
                ratelaw = candidate.function
            else:
                print(f"\tProcessing {candidate.__str__()[10:-19]} Law....")
                ratelaw = candidate

            conc = self.c(self.time).T
            n_param = num_params(ratelaw, conc)
            x0 = np.random.rand(n_param)
            bnds = []
            for i in range(n_param):
                bnds.append((0, None))
            optim = sc.optimize.minimize(
                self.eb_objective_function,
                x0,
                args=(reaction_extent, ratelaw, self.time),
                method=solver,
                bounds=bnds,
            )
            results.append(optim)

        return results

    def eb_ci_parallelise(self, reaction_extent, ratelaw, n_param, solver):
        """
        Runs parameter estimation in parallel for bootstrapping in extent-based modeling.

        Args:
            reaction_extent: Experimental reaction extents.
            ratelaw: Rate law function.
            n_param: Number of parameters in the rate law.
            solver: Optimization solver method.

        Returns:
            The optimized parameter values.

        Notes:
            This function is used for parallel computation in bootstrapping for
            extent-based modeling. It performs parameter estimation using a
            subset of the reaction extent data and returns the optimized parameter values.

        """
        idx = np.random.choice(self.time.shape[0], self.time.shape[0], replace=True)
        idx.sort()
        if idx[0] == 0:
            pass
        else:
            idx = idx[:-1]
            idx = np.insert(idx, 0, 0)

        bs_time = self.time[idx]
        bs_extents = reaction_extent[idx]
        x0 = np.random.rand(n_param)
        bnds = []
        for i in range(n_param):
            bnds.append((0, None))
        optim = sc.optimize.minimize(
            self.eb_objective_function,
            x0,
            args=(bs_extents, ratelaw, bs_time),
            method=solver,
            bounds=bnds,
        ).x
        return optim

    def mole_balance_ft(self, y, t, ratelaws):
        """
        Mole balance equation for the simultaneous approach.

        Args:
            y: Concentration vector.
            t: Time.
            ratelaws: List of rate law functions.

        Returns:
            The derivative of the concentration vector.

        Notes:
            This function represents the mole balance equation for the simultaneous
            approach in reaction kinetics. It calculates the derivative of the
            concentration vector based on the rate laws, concentrations, and flow rates.

        """
        c = y / self.V(t)
        rate = np.array([ratelaw(c) for ratelaw in ratelaws])
        m = np.sum(self.Mw @ y)  # current mass
        dydt = (
            (self._N.T @ rate * self.V(t))
            + (self.Win @ self.uin(t))
            - (self.uout(t) * y / m)
        )
        return dydt

    def ft_objective_function(self, K, C, cand_list, num_params):
        """
        Computes the objective function for fine-tuning using the simultaneous approach.

        Args:
            K: Parameter values for the rate laws.
            C: Experimental concentration data.
            cand_list: List of candidate rate laws or rate law functions.
            num_params: Number of parameters for each rate law.

        Returns:
            The value of the objective function.

        Notes:
            This function compares the simulated concentration data with the experimental data.
            It calculates the norm of the difference between the two sets of data
            as the objective function value.

        """
        K_new = []
        temp = 0
        for n_param in num_params:
            K_new.append(K[temp: temp + n_param])
            temp += n_param

        ratelaws = [
            partial(ratelaw.function, K=K_new[idx])
            if isinstance(ratelaw, RateLaw)
            else partial(ratelaw, K=K_new[idx])
            for idx, ratelaw in enumerate(cand_list)
        ]
        sol = odeint(self.mole_balance_ft, self.n0, self.time, args=(ratelaws,))
        return np.linalg.norm(sol - C)

    def ft_est_params(self, cand_list, guess, num_params, solver):
        """
        Estimates the parameters for fine-tuning the model using the simultaneous approach.

        Args:
            cand_list: List of candidate rate laws or rate law functions.
            guess: Initial guess for the parameter values.
            num_params: Number of parameters for each rate law.
            solver: Optimization solver method.

        Returns:
            The optimization result containing the estimated parameter values.

        Notes:
            This function performs parameter estimation by minimizing the objective
            function based on the simultaneous approach. It iterates over the
            candidates and finds the optimal parameter values using the
            specified optimization solver.

        """
        conc = self.c(self.time)
        x0 = guess

        bnds = []
        for i in range(len(x0)):
            bnds.append((0, None))

        if solver is None:
            optim = sc.optimize.minimize(
                self.ft_objective_function,
                x0,
                args=(conc, cand_list, num_params),
                bounds=bnds,
            )
        else:
            optim = sc.optimize.minimize(
                self.ft_objective_function,
                x0,
                args=(conc, cand_list, num_params),
                method=solver,
                bounds=bnds,
            )
        return optim

    def conf_int_extent_based(
        self, reaction_extent, candidate, confidence, bootstraps, n_jobs, solver
    ):
        """
        Computes the confidence interval for parameter estimation based on extent-based modeling.

        Args:
            reaction_extent: Experimental reaction extents.
            candidate: Candidate rate law or rate law function.
            confidence: Confidence level for the interval.
            bootstraps: Number of bootstraps for estimating the interval.
            n_jobs: Number of parallel jobs for bootstrapping.
            solver: Optimization solver method.

        Returns:
            The confidence interval for the estimated parameter values.

        Notes:
            This function performs bootstrapping to estimate the confidence interval for
            the parameter values based on extent-based modeling.
            It uses the specified rate law or rate law function to compute the
            parameter estimates for each bootstrap sample.

        """
        if isinstance(candidate, RateLaw):
            ratelaw = candidate.function
        else:
            ratelaw = candidate

        conc = self.c(self.time)
        n_param = num_params(ratelaw, conc.T)

        optims = []
        optims = Parallel(n_jobs=n_jobs)(
            delayed(self.eb_ci_parallelise)(reaction_extent, ratelaw, n_param, solver)
            for i in range(bootstraps)
        )

        CI = np.percentile(
            optims, [100 * (1 - confidence) / 2, 100 * (1 - (1 - confidence) / 2)]
        )
        return CI

    def interpret_results(
        self, results: list[list], candidates_list, metric, plot=True, top_k=1
    ):
        """
        Interprets the results of parameter estimation by analyzing the metrics and
        selecting the best rate laws.

        Args:
            results: List of optimization results for each set of candidates.
            candidates_list: List of candidate rate laws or rate law functions.
            metric: Metric to evaluate the performance of rate laws (e.g., "rmse", "aic", "aicc").
            plot: Boolean flag indicating whether to plot the metric values.
            top_k: Number of top rate laws to select as the best (default: 1).

        Returns:
            List of indices representing the best rate laws for each reaction.

        Notes:
            This function analyzes the optimization results and evaluates
            the performance of rate laws based on the specified metric.
            It can plot the metric values for visualization.
            The top-k rate laws with the lowest metric values are selected as
            the best rate laws for each reaction.

        """
        # Edited ones
        # Best Rate law
        best = []

        for i in range(len(candidates_list)):
            result = results[i]
            r_strings = [
                candidate.expression
                if isinstance(candidate, RateLaw)
                else candidate.__str__()[10:-19]
                for candidate in candidates_list[i]
            ]  # creates ratelaw name strings

            if metric == "rmse":
                loss = [res.fun for res in result]

            elif metric == "aic":
                Ks = [
                    num_params(candidate.function, self.c(self.time).T)
                    if isinstance(candidate, RateLaw)
                    else num_params(candidate, self.c(self.time).T)
                    for candidate in candidates_list[i]
                ]
                n = self.time.shape[0]
                loss = [
                    ((2 * Ks[idx]) + (n * np.log(res.fun**2)))
                    for idx, res in enumerate(result)
                ]

            elif metric == "aicc":
                Ks = [
                    num_params(candidate.function, self.c(self.time).T)
                    if isinstance(candidate, RateLaw)
                    else num_params(candidate, self.c(self.time).T)
                    for candidate in candidates_list[i]
                ]
                n = self.time.shape[0]
                loss = [
                    (2 * Ks[idx])
                    + (
                        n * np.log(res.fun**2)
                        + (2 * Ks[idx] * (Ks[idx] + 1) / (n - Ks[idx] - 1))
                    )
                    for idx, res in enumerate(result)
                ]

            else:
                raise ValueError(
                    "Metric not supported. Available metrics are rmse, aic and aicc"
                )

            if plot is True:
                # if plotly not there use matplotlib (TO DO)
                fig = px.bar(
                    x=r_strings,
                    y=loss,
                    labels={"x": "Rate Law", "y": metric.upper()},
                    title=f"{metric.upper()} vs Rate Law for Reaction {i+1}",
                )
                fig.update_layout(xaxis={"categoryorder": "total descending"})
                fig.show()

            # pick top_k elements if candidates_list[i] has >= top_k ratelaws.
            # else: pick len(candidates_list[i]) ratelaws instead of top_k
            if top_k > len(candidates_list[i]):
                k = len(result)
            else:
                k = top_k

            best_idx = heapq.nsmallest(k, range(len(loss)), loss.__getitem__)
            best.append(best_idx)

        return best

    def estimate_parameters(
        self,
        candidates_list,
        method="rate_based",
        metric="rmse",
        conf_int=False,
        n_jobs=-1,
        confidence=0.95,
        bootstraps=1000,
        plot=True,
        solver="Nelder-Mead",
    ):
        """
        Estimates the parameters of the reaction kinetics based on the specified method.

        Args:
            candidates_list: List of candidate rate laws or rate law functions for each reaction.
            method: Method for parameter estimation ("rate_based" or "extent_based",
                default: "rate_based").
            metric: Metric to evaluate the performance of rate laws (default: "rmse").
            conf_int: Boolean flag indicating whether to compute confidence intervals
                (default: False).
            n_jobs: Number of parallel jobs to run in parallel (default: -1, using all
                available processors).
            confidence: Confidence level for computing confidence intervals (default: 0.95).
            bootstraps: Number of bootstrap iterations for confidence interval computation
                (default: 1000).
            plot: Boolean flag indicating whether to plot the metric values (default: True).
            solver: Solver method for optimization (default: "Nelder-Mead").

        Returns:
            Dictionary containing the best rate laws, estimated parameters, and
                optionally confidence intervals.

        Notes:
            This function estimates the parameters of the reaction kinetics using
            either the rate-based or extent-based approach. It evaluates the performance
            of different rate laws using the specified metric and selects the best rate laws.
            If conf_int is True, it computes confidence intervals for the estimated
            parameters using bootstrapping. The function returns a dictionary containing
            the best rate laws, estimated parameters, and (optionally) confidence intervals.
        """
        best_result = {}
        best_result = {}
        best_result["best_ratelaws"] = []
        best_result["params"] = []
        best_result["conf_ints"] = []
        if not hasattr(self, "n"):
            raise ValueError(
                "Concentration data not specified. Add concentration \
                    data through add_concentration_data method"
            )

        if len(candidates_list) != self.R:
            warnings.warn(
                f"candidates_list provided is not a list with a first dimension \
                    size of {self.R}. Please ensure that the list you are \
                        passing has R elements in its first dimension. \
                            Estimating only {len(candidates_list)} reaction kinetics.",
                stacklevel=2,
            )

        if method == "rate_based":
            results = []
            if self.methodology == 1 or self.methodology == "rate":
                reaction_rates = self.reaction_rate_r1()
                for i in range(len(candidates_list)):
                    print(f"Processing Reaction {i+1}:")
                    candidates = candidates_list[i]
                    results.append(self.rb_est_params(reaction_rates[:, i], candidates))

                best_idxs = self.interpret_results(
                    results, candidates_list, metric, plot, top_k=1
                )

                for i in range(len(candidates_list)):
                    r_strings = [
                        candidate.expression
                        if isinstance(candidate, RateLaw)
                        else candidate.__str__()[10:-19]
                        for candidate in candidates_list[i]
                    ]  # creates ratelaw name strings
                    print(f"Reaction {i+1}: ")
                    print(f"Best Rate Law: {r_strings[best_idxs[i][0]]}")
                    print(f"Estimated Parameters: {results[i][best_idxs[i][0]].x}")
                    best_candidate = candidates_list[i][best_idxs[i][0]]
                    best_result["best_ratelaws"].append(best_candidate)
                    best_result["params"].append(results[i][best_idxs[i][0]].x)
                    if conf_int:
                        ci = self.conf_int_rate_based(
                            reaction_rates[:, i],
                            best_candidate,
                            confidence,
                            bootstraps,
                            n_jobs,
                        )
                        print(f"Confidence Interval {ci}")
                        best_result["conf_ints"].append(ci)
                    print()
                best_result["results"] = results

        elif method == "extent_based":
            results = []
            if self.methodology == 1:
                q0T, S0T, M0T, Q0T = compute_matrices(self.N, self.Win, self.n0)
                n = self.n(self.time)
                self._xr = n @ S0T.T
                self._xin = n @ M0T.T
                self._lamda = n @ q0T.T
                reaction_extents = self._xr

            elif self.methodology == 2:
                self._xin, self._lamda, _ = compute_inlet_extents(
                    self.uin, self.uout, self.time, self.n0, self.Mw
                )
                self._xr = np.linalg.pinv(self.Na.T) @ (
                    self.n(self.time).T
                    - (self.Wina @ self._xin.T)
                    - ((self._lamda * self.n0a).T)
                )
                self._xr = self._xr.T
                reaction_extents = self._xr
                n = (
                    (self._xr @ self.N)
                    + (self._xin @ self.Win.T)
                    + (self._lamda * self.n0)
                )
                c = (n.T / self.V(self.time)).T
                self.add_concentration_data(c, self.time)

            for i in range(len(candidates_list)):
                print(f"Processing Reaction {i+1}:")
                candidates = candidates_list[i]
                results.append(
                    self.eb_est_params(reaction_extents[:, i], candidates, solver)
                )

            best_idxs = self.interpret_results(
                results, candidates_list, metric, plot, top_k=1
            )

            for i in range(len(candidates_list)):
                r_strings = [
                    candidate.expression
                    if isinstance(candidate, RateLaw)
                    else candidate.__str__()[10:-19]
                    for candidate in candidates_list[i]
                ]  # creates ratelaw name strings
                print(f"Reaction {i+1}: ")
                print(f"Best Rate Law: {r_strings[best_idxs[i][0]]}")
                print(f"Estimated Parameters: {results[i][best_idxs[i][0]].x}")
                best_candidate = candidates_list[i][best_idxs[i][0]]
                best_result["best_ratelaws"].append(best_candidate)
                best_result["params"].append(results[i][best_idxs[i][0]].x)
                if conf_int:
                    ci = self.conf_int_extent_based(
                        reaction_extents[:, i],
                        best_candidate,
                        confidence,
                        bootstraps,
                        n_jobs,
                        solver,
                    )
                    print(f"Confidence Interval {ci}")
                    best_result["conf_ints"].append(ci)
                print()
            best_result["results"] = results

        else:
            raise ValueError(
                f"Valid methods are rate-based and extent-based. \
                    Unsupported method {method} provided"
            )

        self.results = results
        self.candidates_list = candidates_list
        self.metric = metric
        self.best_result = best_result
        return best_result

    def finetune(self, top_k=2, metric=None, solver="Nelder-Mead"):
        """
        Fine-tunes the model by selecting the best rate laws from the estimated parameters.

        Args:
            top_k: Number of top rate laws to consider for fine-tuning (default: 2).
            metric: Metric to evaluate the performance of rate laws (default: None,
            uses the metric from previous estimation).
            solver: Solver method for optimization (default: "Nelder-Mead").

        Returns:
            Dictionary containing the best rate laws and their estimated parameters.

        Notes:
            This method selects the best rate laws from the estimated parameters obtained
            using the `estimate_parameters` method. It considers the top_k rate laws
            and evaluates their performance using the specified metric.
            The method returns a dictionary containing the best rate laws and
            their estimated parameters.
        """
        best_result = {}
        best_result["best_ratelaws"] = []
        best_result["params"] = []
        candidates_list = self.candidates_list
        self._N = self.N[: len(candidates_list), :]  # for fine tuning just in case
        tuned_results = []
        if metric is None:
            metric = self.metric

        if not hasattr(self, "results"):
            raise ValueError("Run estimate parameters method before model finetuning")

        best_idxs = self.interpret_results(
            self.results, self.candidates_list, metric, plot=False, top_k=top_k
        )
        combinations = list(itertools.product(*best_idxs))
        for c in combinations:
            cand_list = []
            guess = []
            n_params = []
            # get ratelaws corresponding to the candidates
            for i in range(len(c)):
                if isinstance(self.candidates_list[i][c[i]], RateLaw):
                    cand_list.append(self.candidates_list[i][c[i]])
                else:
                    cand_list.append(self.candidates_list[i][c[i]])

                # values found in previous method is used as initial guesses
                guess.extend(list(self.results[i][c[i]].x))
                n_params.append(len(list(self.results[i][c[i]].x)))

            tuned_results.append(self.ft_est_params(cand_list, guess, n_params, solver))

        if metric == "rmse":
            loss = [res.fun for res in tuned_results]

        elif metric == "aic":
            Ks = np.array(
                [
                    [
                        num_params(candidates_list[i][c[i]], self.c(self.time).T)
                        if not isinstance(candidates_list[i][c[i]], RateLaw)
                        else num_params(
                            candidates_list[i][c[i]].function, self.c(self.time).T
                        )
                        for i in range(len(c))
                    ]
                    for c in combinations
                ]
            )
            Ks = np.sum(Ks, axis=1)
            n = self.time.shape[0]
            loss = [
                ((2 * Ks[idx]) + (n * np.log(res.fun**2)))
                for idx, res in enumerate(tuned_results)
            ]

        elif metric == "aicc":
            Ks = np.array(
                [
                    [
                        num_params(candidates_list[i][c[i]], self.c(self.time).T)
                        if not isinstance(candidates_list[i][c[i]], RateLaw)
                        else num_params(
                            candidates_list[i][c[i]].function, self.c(self.time).T
                        )
                        for i in range(len(c))
                    ]
                    for c in combinations
                ]
            )
            Ks = np.sum(Ks, axis=1)
            n = self.time.shape[0]
            loss = [
                (2 * Ks[idx])
                + (
                    n * np.log(res.fun**2)
                    + (2 * Ks[idx] * (Ks[idx] + 1) / (n - Ks[idx] - 1))
                )
                for idx, res in enumerate(tuned_results)
            ]

        else:
            raise ValueError(
                "Metric not supported. Available metrics are rmse, aic and aicc"
            )

        best_idx = heapq.nsmallest(1, range(len(loss)), loss.__getitem__)
        best_combination = combinations[best_idx[0]]

        print("Best RateLaws: ")
        n_param = [
            len(list(self.results[i][c].x)) for i, c in enumerate(best_combination)
        ]
        temp = 0
        for idx, c in enumerate(best_combination):
            if isinstance(self.candidates_list[idx][c], RateLaw):
                print(
                    f"For reaction {idx+1}, best ratelaw is \
                        {self.candidates_list[idx][c].expression}"
                )
                print(
                    f"Parameters estimated: {tuned_results[best_idx[0]].x[temp:temp+n_param[idx]]}"
                )
                best_result["best_ratelaws"].append(
                    self.candidates_list[idx][c].function
                )
                best_result["params"].append(
                    tuned_results[best_idx[0]].x[temp: temp + n_param[idx]]
                )
            else:
                print(
                    f"For reaction {idx+1}, best ratelaw is \
                        {self.candidates_list[idx][c].__str__()[10:-19]}"
                )
                print(
                    f"Parameters estimated: {tuned_results[best_idx[0]].x[temp:temp+n_param[idx]]}"
                )
                best_result["best_ratelaws"].append(self.candidates_list[idx][c])
                best_result["params"].append(
                    tuned_results[best_idx[0]].x[temp: temp + n_param[idx]]
                )
            temp += n_param[idx]

        best_result["tuned_results"] = tuned_results
        self.best_result = best_result
        return best_result

    def simulate_best_rl(self, plot=True):
        """
        Simulates the best rate laws obtained from model finetuning.

        Args:
            plot: Boolean flag indicating whether to plot the simulation results (default: True).

        Returns:
            Dictionary containing the simulation results.

        Notes:
            This method simulates the best rate laws obtained from model finetuning.
            It adds the rate laws and their estimated parameters to the model and
            runs the simulation.
            The method returns a dictionary containing the simulation results.
            If plot is True, it also plots the concentration profiles over time.
        """
        if not hasattr(self, "best_result"):
            raise ValueError(
                "Run estimate parameters method before simulating best ratelaws"
            )

        act_ratelaws = self.best_result["best_ratelaws"]
        K = self.best_result["params"]
        # sim = Simulate(self.N, self.Mw, self.V, self.Winhat, self.uin, self.uout, self.n0)
        self.add_ratelaws(act_ratelaws, K)
        results_normal = self.run_simulation(self.time, alpha=0)

        if plot:
            c = results_normal["moles"].shape[1]
            for i in range(c):
                plt.plot(self.time, results_normal["moles"][:, i], label=str(i))
            plt.xlabel("Time [min]")
            plt.ylabel("Conc [mol L-1]")
            plt.title("fitted plot")
            plt.legend()
            plt.show()
        return results_normal

# Examples

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
# from Incremental import Incremental
# from ratelawgen import CandidateRateLaws, RateLaw
# from simulate import Simulate
# from preprocess import read_file

## Example 1

## Example 2

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
# from Incremental import Incremental
# from ratelawgen import CandidateRateLaws, RateLaw
# from simulate import Simulate
# from preprocess import read_file

qin = 0.3
cin = np.array([0, 6, 0, 0, 0, 0])
N = [[-1, -1, 1, 0, 0, 0], [0, -2, 0, 1, 0 ,0], [0, -1, 0, 0, 1, 0]]
n0 = np.array([1, 1, 1, 1, 1, 1])
Mw = np.diag([67,84,151,168,84,84+151])/1000
Winhat = [0, 1, 0, 0, 0, 0]
uin = np.sum(qin * Mw@cin )
uin=0.1512
uout = 0.1512
V = 1
time = np.linspace(0,50,151)

cand_ratelaws1 = CandidateRateLaws(N[0], type = "Irreversible")
cand_ratelaws2 = CandidateRateLaws(N[1], type = "Irreversible")
cand_ratelaws3 = CandidateRateLaws(N[2], type = "Irreversible")

candidates_list = [[cand_ratelaws1], [cand_ratelaws2], [cand_ratelaws3]]

cand_list = []
type_list = ["Irreversible","Irreversible","Irreversible"]
for i in range(3):
    cand_list += [CandidateRateLaws(N[i], type = type_list[i])]

# Creating Model Object
model = Incremental(N, Mw, V, Winhat, uin, uout, n0)
# Adding concentration data from csv file
model.add_concentration_data("aceto_pyrrole_a_missing.csv")

# Rate based method not working
#res = model.estimate_parameters(candidates_list, method = 'extent_based', conf_int = True, metric = 'aicc', solver = None)
res = model.estimate_parameters(cand_list, method = 'extent_based', metric = 'aicc', solver = None)
#res = model.estimate_parameters(cand_list, method = 'rate_based', conf_int = True, metric = 'aicc', plot = True, bootstraps = 1000) #

## Example 3

In [ ]:
qin = 0

N = [[-1, -1,  1,  1,  0,  0],[ 0, -1, -1,  1,  1,  0],[ 0, -1,  0,  1, -1,  1]]
n0 = np.array([1, 1, 1, 1, 1, 1])
Mw = np.diag([67,84,151,168,84,84+151])/1000
Winhat = [0.25,0.25,0,0,0,0]
uin = 0
uout = 0
V = 1
time = np.linspace(0.0, 3.0, 31)

cand_ratelaws1 = CandidateRateLaws(N[0], type = "Irreversible")
cand_ratelaws2 = CandidateRateLaws(N[1], type = "Irreversible")
cand_ratelaws3 = CandidateRateLaws(N[2], type = "Irreversible")

candidates_list = [[cand_ratelaws1], [cand_ratelaws2], [cand_ratelaws3]]

cand_list = []
type_list = ["Irreversible","Irreversible","Irreversible"]
for i in range(3):
    cand_list += [CandidateRateLaws(N[i], type = type_list[i])]

# Creating Model Object
model = Incremental(N, Mw, V, Winhat, uin, uout, n0)
# Adding concentration data from csv file
model.add_concentration_data("biodiesel_conc_data.csv")

# Rate based method not working
#res = model.estimate_parameters(candidates_list, method = 'extent_based', conf_int = True, metric = 'aicc', solver = None)
res = model.estimate_parameters(cand_list, method = 'extent_based', metric = 'aicc', solver = None)

Enter reactor configuration as a number. Permitted entries: 1 = Batch; 2 = Semi-Batch; 3 = CSTR.
Enter reactor configuration:1
Entered reactor configuration is: 1


**Data for reactor configuration received successfully.**

Enter the number of reactions in the reaction system:3
Entered value of number of reactions is: 3


**Data for number of reactions received successfully.**

Enter the number of species in the reaction system:6
Entered value of number of species is: 6


...

**Data regarding batch reactor volume variability received successfully.**

Enter batch reactor volume in SI units. (Enter 1 if reactor volume is unknown):0.001
Entered reactor volume is: 0.001


**Data for initial volume received successfully.**

Processing Reaction 1:
	Processing expression K[0]
	Processing expression K[0]*C[1]^1
	Processing expression K[0]*C[0]^1*C[1]^1
	Processing expression K[0]*C[0]^1
Processing Reaction 2:
	Processing expression K[0]
	Processing expression K[0]*C[2]^1
	Processing expression K[0]*C[1]^1*C[2]^1
	Processing expression K[0]*C[1]^1
Processing Reaction 3:
	Processing expression K[0]
	Processing expression K[0]*C[4]^1
	Processing expression K[0]*C[1]^1*C[4]^1
	Processing expression K[0]*C[1]^1


Reaction 1: 
Best Rate Law: K[0]*C[1]^1
Estimated Parameters: [0.57221109]

Reaction 2: 
Best Rate Law: K[0]*C[1]^1*C[2]^1
Estimated Parameters: [5.59739384]

Reaction 3: 
Best Rate Law: K[0]*C[1]^1*C[4]^1
Estimated Parameters: [12.48100341]



In [ ]:
qin = 0

N = [[-1, -1,  1,  0],
        [-1,  0, -1,  1]]
n0 = np.array([ 4.70927436e-01,  3.12088735e-01, -1.84177676e-02,
        -1.67785825e-03])
Mw = np.diag([94,78.3,136,36.3])/1000
Winhat = [0,0,0,0]
uin = 0
uout = 0
V = 1
time = np.linspace(0.0, 10.0, 31)

cand_ratelaws1 = CandidateRateLaws(N[0], type = "Irreversible")
cand_ratelaws2 = CandidateRateLaws(N[1], type = "Irreversible")

candidates_list = [[cand_ratelaws1], [cand_ratelaws2]]

cand_list = []
type_list = ["Irreversible","Irreversible"]
for i in range(2):
    cand_list += [CandidateRateLaws(N[i], type = type_list[i])]

# Creating Model Object
model = Incremental(N, Mw, V, Winhat, uin, uout, n0)
# Adding concentration data from csv file
model.add_concentration_data('/content/p-1 data case 3 210c.csv')

Enter reactor configuration as a number. Permitted entries: 1 = Batch; 2 = Semi-Batch; 3 = CSTR.
Enter reactor configuration:1
Entered reactor configuration is: 1


**Data for reactor configuration received successfully.**

Enter the number of reactions in the reaction system:2
Entered value of number of reactions is: 2


**Data for number of reactions received successfully.**

Enter the number of species in the reaction system:4
Entered value of number of species is: 4


...

**Reacting species have been assigned the following names: ['A', 'B', 'C', 'D'] in the same order in which they appear in the columns of the stoichiometric matrix.**

Enter reaction reversibility information as a one-dimensional list. Enter 0 for irreversible reaction and 1 for
reversible reaction. For reaction system with 2 reactions, where reaction 1 is irreversible and reaction 2 is
reversible, enter [0,1]. The order of reactions should be the same as the order in which they appear in the rows of the
stoichiometric matrix.
Enter reaction reversibility information:[0,0]
User input is: [0,0]


**Data for reaction reversibility received successfully.**

**• The following is pertinent information regarding the concentration data file structure.**

• The first row in the file must contain column headers.
• Column 1 header can have any name.
• Other column headers (columns 2, 3, . . . ) must have species names.
• Allowed species names are: ['A', 'B', 'C', 'D']
• The first column in the file must contain time points at which concentration data is collected.
• The first time point must be zero.
• The remaining columns pertain to concentration data of participating species.
• The file can be a csv file or a tab-separated text file.
• All data should be in SI units.
Upload concentration data file to local storage and enter concentration data file name.
Enter concentration data file name:data.csv
Entered file name is: data.csv
Reactor configuration is Batch → Wina and n0a are not required for reconstruction.
One or more species names in concentration data file are invalid. Allowed names: A, B, C, D. Provided: A, C, D, Unnamed: 2
